In [ ]:
import os
print("Current working directory:", os.getcwd())

In [2]:
"""
Run Simple Parallel Scan for Test Bar

This script runs the test bar scan across multiple exchanges in parallel
using a simplified parallel scanning approach that avoids console output issues.
"""

import asyncio
import sys
import os
import logging

sys.path.insert(0, os.getcwd())

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

print(f"✓ Added {os.getcwd()} to sys.path")

# Import the simple parallel scanner
from run_parallel_scanner import run_parallel_exchanges, run_parallel_multi_timeframes_all_exchanges, sf_exchanges_1w
from scanner.main import run_scanner, kline_cache

# Define exchanges
futures_exchanges = ["binance_futures", "bybit_futures", "mexc_futures", "gateio_futures"]
spot_exchanges = ["binance_spot", "bybit_spot", "kucoin_spot", "mexc_spot", "gateio_spot"]
spot_exchanges_1w = ["binance_spot", "bybit_spot", "gateio_spot"] + sf_exchanges_1w
fast_exchanges = ["binance_futures", "binance_spot", "bybit_spot", "gateio_spot"]
slow_exchanges = ["kucoin_spot", "mexc_spot", "mexc_futures"]

async def main():
    # Clear cache for fresh data
    kline_cache.clear()
    
    """    
    # Run parallel scan for test bar strategy on spot exchanges
    result = await run_parallel_exchanges(
        timeframe="4h",                    # Example timeframe
        strategies=["hbs_breakout", "test_bar", "consolidation_breakout", "sma50_breakout", "trend_breakout", "pin_up"],
        # strategies=["reversal_bar"],       
        exchanges=spot_exchanges,          # Spot exchanges to scan
        users=["default"],                 # Recipients for Telegram notifications
        send_telegram=True,                # Enable Telegram notifications
        min_volume_usd=None,               # Use default volume threshold
        save_to_csv=True,                   # Enable saving to CSV
        check_bar="both"                   # NEW PARAMETER: "last_closed", "current", or "both"
    )
    """
    # Run multi-timeframe parallel scan
    result = await run_parallel_multi_timeframes_all_exchanges(
        timeframes=["1w"],     # Multiple timeframes0
        strategies= ["hbs_breakout"], #["confluence", "consolidation_breakout", "channel_breakout", "sma50_breakout", "loaded_bar", "pin_up", "trend_breakout"], #"engulfingReversal", "vs_wakeup],        # Strategies to scan
        exchanges= ["binance_spot"], #fast_exchanges,          # Exchanges to scan
        users=["default"],                 # Recipients for notifications
        send_telegram=True,                # Enable notifications
        min_volume_usd=None,               # Use default volume threshold
        save_to_csv=False,                   # Enable saving to CSV
        check_bar="current"                   # NEW PARAMETER: "last_closed", "current", or "both"
    )
     #"""
    
    print("Scan completed!")
    return result

if __name__ == "__main__":
    asyncio.run(main())


  RUNNING PARALLEL MULTI-TIMEFRAME SCAN ON ALL EXCHANGES

• Exchanges: binance_spot
• Timeframes: 1w
• Strategies: hbs_breakout
• Check bar: current
• Notifications: Enabled
• Recipients: default
• Save to CSV: Disabled
• Start time: 14:15:53

Fetching market data...

Processing timeframe: 1w
1w: 1 FAST, 0 SLOW exchanges

  PHASE: FAST 1w (1 exchanges)

[14:15:53] Starting scan on binance_spot for 1w timeframe (check_bar=current)...


✓ Added /home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project to sys.path


Found 421 markets on Binance Spot for 1w timeframe
Processing 421 symbols with parallel strategies (batch size: 25)
[14:16:51] ✓ Completed binance_spot scan: 0 signals found

  PHASE: SLOW 1w (0 exchanges)

No exchanges in this phase.

  PARALLEL MULTI-TIMEFRAME MULTI-EXCHANGE SCAN RESULTS

Total signals found across all exchanges and timeframes: 0
Start time: 14:15:53
End time: 14:16:51
Duration: 0:00:58


Scan completed!


In [4]:
# Remove the problematic nested directory completely
import shutil
import os

problematic_path = '/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project/Project'
if os.path.exists(problematic_path):
    shutil.rmtree(problematic_path)
    print(f"Removed problematic directory: {problematic_path}")

# Clear sys.path of any references to the nested directory
import sys
sys.path = [p for p in sys.path if 'Project/Project' not in p]

# Ensure only the correct path is in sys.path
correct_path = '/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project'
if correct_path not in sys.path:
    sys.path.insert(0, correct_path)

print("Fixed path structure. Restart kernel now.")

Removed problematic directory: /home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025/Project/Project
Fixed path structure. Restart kernel now.


In [ ]:
#Verify OHLCV data from SF

from exchanges.sf_pairs_service import SFPairsService
import pandas as pd

sf_service = SFPairsService()

symbol = "BTC"
quote = "USDT"
exchange = "Kucoin"   # or "Mexc"
timeframe = "1w"      # weekly
limit = 2             # how many candles to fetch

# Fetch OHLCV data
raw_data = sf_service.get_ohlcv_for_pair(symbol, quote, exchange, timeframe, limit)

# Convert to DataFrame
df = pd.DataFrame(raw_data)

# Normalize datetime
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')
elif 'time' in df.columns:
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df = df.set_index('time')

# Keep only required columns
df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

# Print with timeframe in console
print(f"\n📊 {symbol}/{quote} OHLCV ({timeframe}) on {exchange}")
print(df.tail(1))  # last row = current bar


In [ ]:
# Binance BTC dominated pairs - Confluence Scanner with Direct API

from telegram.ext import Application
import logging
import pandas as pd
import numpy as np
import asyncio
import nest_asyncio
from datetime import datetime
from tqdm.asyncio import tqdm
import sys
import os
import html
import aiohttp
import time

# Add project path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)

from custom_strategies import detect_confluence

class BinanceBTCConfluenceScanner:
    def __init__(self, telegram_token, telegram_chat_id, timeframe, offset=1):
        self.telegram_token = telegram_token
        self.telegram_chat_id = telegram_chat_id
        self.telegram_app = None
        self.exchange = "Binance"
        self.timeframe = timeframe
        self.quote_currency = "BTC"
        self.offset = offset
        self.base_url = "https://api.binance.com"
        self.session = None
        
        # Binance timeframe mapping - All available intervals
        self.timeframe_map = {
            # Minutes
            "1m": "1m",
            "3m": "3m", 
            "5m": "5m",
            "15m": "15m",
            "30m": "30m",
            # Hours
            "1h": "1h",
            "2h": "2h",
            "4h": "4h",
            "6h": "6h",
            "8h": "8h",
            "12h": "12h",
            # Days
            "1d": "1d",
            "2d": "2d",
            "3d": "3d",
            # Weeks/Months
            "1w": "1w",
            "1M": "1M"
        }
        
    async def init_session(self):
        """Initialize aiohttp session"""
        if self.session is None:
            self.session = aiohttp.ClientSession()
            
    async def close_session(self):
        """Close aiohttp session"""
        if self.session:
            await self.session.close()
            self.session = None
        
    async def init_telegram(self):
        if self.telegram_app is None:
            self.telegram_app = Application.builder().token(self.telegram_token).build()

    async def get_btc_pairs(self):
        """Get all BTC trading pairs from Binance"""
        await self.init_session()
        
        try:
            url = f"{self.base_url}/api/v3/exchangeInfo"
            async with self.session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    
                    # Filter for BTC pairs that are actively trading
                    btc_pairs = []
                    for symbol_info in data['symbols']:
                        if (symbol_info['quoteAsset'] == 'BTC' and 
                            symbol_info['status'] == 'TRADING' and
                            symbol_info['isSpotTradingAllowed']):
                            
                            btc_pairs.append({
                                'symbol': symbol_info['symbol'],
                                'baseAsset': symbol_info['baseAsset'],
                                'quoteAsset': symbol_info['quoteAsset']
                            })
                    
                    return btc_pairs
                else:
                    logging.error(f"Error fetching exchange info: {response.status}")
                    return []
        except Exception as e:
            logging.error(f"Error fetching BTC pairs: {str(e)}")
            return []

    async def get_klines(self, symbol, interval, limit=100):
        """Get kline/candlestick data from Binance"""
        await self.init_session()
        
        try:
            url = f"{self.base_url}/api/v3/klines"
            params = {
                'symbol': symbol,
                'interval': interval,
                'limit': limit
            }
            
            async with self.session.get(url, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    
                    # Convert to DataFrame
                    df = pd.DataFrame(data, columns=[
                        'open_time', 'open', 'high', 'low', 'close', 'volume',
                        'close_time', 'quote_asset_volume', 'number_of_trades',
                        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                    ])
                    
                    # Convert timestamps to datetime
                    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
                    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
                    
                    # Convert OHLCV to numeric
                    for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume']:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                    
                    # Set datetime index
                    df.set_index('open_time', inplace=True)
                    
                    # Select only OHLCV columns needed for confluence
                    # Note: Using quote_asset_volume as it's the volume in BTC
                    result_df = df[['open', 'high', 'low', 'close', 'quote_asset_volume']].copy()
                    result_df.rename(columns={'quote_asset_volume': 'volume'}, inplace=True)
                    
                    return result_df
                    
                elif response.status == 429:
                    # Rate limit hit, wait a bit
                    await asyncio.sleep(1)
                    return None
                else:
                    return None
                    
        except Exception as e:
            logging.error(f"Error fetching klines for {symbol}: {str(e)}")
            return None

    async def send_telegram_alert(self, results):
        if not results:
            return
            
        try:
            message = f"🚨 Confluence Detection - Binance BTC Pairs {self.timeframe}\n\n"
            
            # Map timeframe to TradingView format - Extended mapping
            tv_timeframe_map = {
                # Minutes
                "1m": "1", "3m": "3", "5m": "5", "15m": "15", "30m": "30",
                # Hours  
                "1h": "60", "2h": "120", "4h": "240", "6h": "360", "8h": "480", "12h": "720",
                # Days
                "1d": "1D", "2d": "2D", "3d": "3D",
                # Weeks/Months
                "1w": "1W", "1M": "1M"
            }
            tv_timeframe = tv_timeframe_map.get(self.timeframe.lower(), self.timeframe)
            
            for result in results:
                formatted_symbol = result['symbol']
                tv_link = f"https://www.tradingview.com/chart/?symbol=BINANCE:{formatted_symbol}&interval={tv_timeframe}"
                
                # Escape HTML entities in the URL
                escaped_link = html.escape(tv_link)
                
                # Format according to BTC specifications
                time_str = ""
                if result.get('timestamp') is not None:
                    time_str = f"Time: {result['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\n"
                
                message += (
                    f"Symbol: {result['symbol']}\n"
                    f"{time_str}"
                    f"Volume BTC: ₿{result['volume_btc']:,.4f}\n"
                    f"Close: <a href='{escaped_link}'>₿{result['close']:.8f}</a>\n"
                    f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                    f"Close Off Low: {result['close_off_low']:.1f}%\n"
                    f"Momentum: {result['momentum_score']:.4f}\n"
                    f"{'='*30}\n"
                )
            
            # Split message more carefully to avoid breaking HTML tags
            max_length = 4000
            
            if len(message) > max_length:
                # Split at natural breaks (between results) to avoid breaking HTML
                sections = message.split('='*30 + '\n')
                current_chunk = ""
                
                for section in sections:
                    if len(current_chunk + section + '='*30 + '\n') > max_length:
                        if current_chunk:
                            await self.telegram_app.bot.send_message(
                                chat_id=self.telegram_chat_id,
                                text=current_chunk.strip(),
                                parse_mode='HTML',
                                disable_web_page_preview=True
                            )
                        current_chunk = section + '\n'
                    else:
                        current_chunk += section + '='*30 + '\n'
                
                # Send remaining chunk
                if current_chunk.strip():
                    await self.telegram_app.bot.send_message(
                        chat_id=self.telegram_chat_id,
                        text=current_chunk.strip(),
                        parse_mode='HTML',
                        disable_web_page_preview=True
                    )
            else:
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=message,
                    parse_mode='HTML',
                    disable_web_page_preview=True
                )
                
        except Exception as e:
            logging.error(f"Error sending Telegram alert: {str(e)}")
            
            # Fallback: send without HTML formatting
            try:
                simple_message = f"🚨 Confluence Detection - Binance BTC Pairs {self.timeframe}\n\n"
                for result in results:
                    simple_message += (
                        f"Symbol: {result['symbol']}\n"
                        f"Volume BTC: ₿{result['volume_btc']:,.4f}\n"
                        f"Close: ₿{result['close']:.8f}\n"
                        f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                        f"Components: Vol={result['high_volume']}, Spread={result['spread_breakout']}, Mom={result['momentum_breakout']}\n\n"
                    )
                
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=simple_message,
                    disable_web_page_preview=True
                )
            except Exception as fallback_error:
                logging.error(f"Fallback Telegram send also failed: {str(fallback_error)}")

    def scan_single_market(self, pair, df):
        """Scan a single market for Confluence pattern in the specified bar"""
        try:
            if df is None or len(df) < 50:  # Need enough data for confluence
                return None
            
            # Calculate which bar to check based on offset
            check_bar = -(self.offset + 1)  # offset=0 means current bar (-1), offset=1 means last closed (-2), etc.
            
            # Run confluence detection
            detected, result = detect_confluence(df, check_bar=check_bar)
            
            if detected:
                # Get the target bar values
                target_close = df['close'].iloc[check_bar]
                target_volume = df['volume'].iloc[check_bar]  # This is already in BTC
                
                confluence_result = {
                    'symbol': pair['symbol'],
                    'volume_btc': float(target_volume),
                    'close': float(target_close),
                    'volume': float(target_volume),
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                    'bar_range': result['bar_range'],
                    'timestamp': df.index[check_bar] if hasattr(df.index, '__getitem__') else None
                }
                return confluence_result
                
        except Exception as e:
            logging.error(f"Error processing {pair['symbol']}: {str(e)}")
        return None

    async def scan_all_markets(self):
        """Scan all BTC markets for Confluence pattern"""
        await self.init_telegram()
        
        try:
            # Define volume thresholds for BTC pairs based on timeframe
            volume_thresholds = {
                # Minutes - higher volume needed for shorter timeframes
                "1m": 0.01,   "3m": 0.02,   "5m": 0.03,
                "15m": 0.05,  "30m": 0.08,
                # Hours
                "1h": 0.1,    "2h": 0.15,   "4h": 0.2,
                "6h": 0.25,   "8h": 0.3,    "12h": 0.35,
                # Days
                "1d": 0.4,    "2d": 0.8,    "3d": 1.2,
                # Weeks/Months
                "1w": 2.0,    "1M": 8.0
            }
            min_volume = volume_thresholds.get(self.timeframe.lower(), 0.1)  # Default 0.1 BTC
            
            # Create offset description
            if self.offset == 0:
                offset_desc = "current candle"
            elif self.offset == 1:
                offset_desc = "last closed candle"
            else:
                offset_desc = f"{self.offset} candles ago"
            
            print(f"Scanning Binance BTC pairs for Confluence patterns in {offset_desc}...")
            print(f"Timeframe: {self.timeframe}")
            print(f"Minimum volume threshold: ₿{min_volume:.2f}")
            
            # Get all BTC pairs from Binance
            print("Fetching BTC pairs from Binance...")
            btc_pairs = await self.get_btc_pairs()
            
            if not btc_pairs:
                print("No BTC pairs found or error fetching pairs")
                return []
            
            print(f"Found {len(btc_pairs)} BTC pairs to scan")
            
            # Filter out stablecoins and obvious non-trading pairs
            filtered_pairs = []
            skip_tokens = ['USDT', 'USDC', 'BUSD', 'DAI', 'TUSD', 'USDD', 'FDUSD']
            
            for pair in btc_pairs:
                if pair['baseAsset'] not in skip_tokens:
                    filtered_pairs.append(pair)
            
            print(f"After filtering: {len(filtered_pairs)} pairs to scan")
            
            # Get Binance timeframe - fallback to 1d if not found
            binance_interval = self.timeframe_map.get(self.timeframe.lower(), "1d")
            
            # Process all pairs with progress bar
            all_results = []
            successful_scans = 0
            
            with tqdm(total=len(filtered_pairs), desc="Scanning markets") as pbar:
                for pair in filtered_pairs:
                    try:
                        # Get OHLCV data from Binance
                        df = await self.get_klines(pair['symbol'], binance_interval, 100)
                        
                        if df is None or len(df) < 50:
                            pbar.update(1)
                            continue
                        
                        successful_scans += 1
                        target_idx = -(self.offset + 1)
                        
                        # Update progress bar with current symbol
                        pbar.set_description(f"Scanning: {pair['symbol']} ({len(df)} candles)")
                        
                        # Check volume threshold for the target candle
                        try:
                            target_candle_volume = float(df['volume'].iloc[target_idx])  # Already in BTC
                            
                            # Only process if volume meets threshold
                            if target_candle_volume >= min_volume:
                                result = self.scan_single_market(pair, df)
                                if result:
                                    all_results.append(result)
                                    print(f"Found Confluence: {pair['symbol']} 🎯")
                        except (IndexError, ValueError):
                            pass  # Skip if we can't calculate volume
                        
                        # Add small delay to respect rate limits
                        await asyncio.sleep(0.1)
                                
                    except Exception as e:
                        if "429" in str(e):
                            # Rate limit - add longer delay
                            await asyncio.sleep(2)
                    finally:
                        pbar.update(1)
            
            print(f"Successfully scanned {successful_scans}/{len(filtered_pairs)} pairs")
            
            # Sort by volume
            all_results.sort(key=lambda x: x['volume_btc'], reverse=True)
            
            # Send Telegram alert if we found any patterns
            if all_results:
                await self.send_telegram_alert(all_results)
            
            return all_results
            
        except Exception as e:
            logging.error(f"Error scanning markets: {str(e)}")
            return []
        finally:
            await self.close_session()

async def run_binance_btc_confluence_scanner(timeframe, offset=1):
    """
    Run the Binance BTC Confluence scanner
    
    Parameters:
    timeframe (str): Time period - Available options:
                    Minutes: 1m, 3m, 5m, 15m, 30m
                    Hours: 1h, 2h, 4h, 6h, 8h, 12h  
                    Days: 1d, 2d, 3d
                    Weeks/Months: 1w, 1M
    offset (int): Bar offset (0=current, 1=last closed, 2=two bars ago, etc.)
    """
    
    if offset == 0:
        offset_desc = "current candle"
    elif offset == 1:
        offset_desc = "last closed candle"
    else:
        offset_desc = f"{offset} candles ago"
    
    print(f"Starting Binance BTC Confluence scan for {offset_desc} on {timeframe}...")
    
    # Use the confluence telegram token
    telegram_token = "8066329517:AAHVr6kufZWe8UqCKPfmsRhSPleNlt_7G-g"
    telegram_chat_id = "375812423"
    
    scanner = BinanceBTCConfluenceScanner(telegram_token, telegram_chat_id, timeframe, offset)
    results = await scanner.scan_all_markets()
    
    if results:
        print(f"\nFound {len(results)} Confluence patterns:")
        
        # Convert results to DataFrame for console display
        df_results = pd.DataFrame(results)
        
        # Round numeric columns for BTC precision
        df_results['volume_btc'] = df_results['volume_btc'].round(4)
        df_results['close'] = df_results['close'].round(8)
        df_results['volume'] = df_results['volume'].round(4)
        df_results['volume_ratio'] = df_results['volume_ratio'].round(2)
        df_results['close_off_low'] = df_results['close_off_low'].round(1)
        df_results['momentum_score'] = df_results['momentum_score'].round(4)
        
        # Reorder columns for better display
        display_cols = ['symbol', 'close', 'volume_btc', 'volume_ratio', 'close_off_low', 
                       'momentum_score', 'high_volume', 'spread_breakout', 'momentum_breakout']
        available_cols = [col for col in display_cols if col in df_results.columns]
        
        # Display the results
        print(df_results[available_cols])
        
        # Show component analysis
        print(f"\n🔧 COMPONENT ANALYSIS:")
        vol_count = df_results['high_volume'].sum()
        spread_count = df_results['spread_breakout'].sum()
        momentum_count = df_results['momentum_breakout'].sum()
        
        print(f"High Volume signals: {vol_count}/{len(results)} ({vol_count/len(results)*100:.1f}%)")
        print(f"Spread Breakout signals: {spread_count}/{len(results)} ({spread_count/len(results)*100:.1f}%)")
        print(f"Momentum Breakout signals: {momentum_count}/{len(results)} ({momentum_count/len(results)*100:.1f}%)")
        
    else:
        print(f"\nNo Confluence patterns found in {offset_desc}")

# Set up logging
logging.basicConfig(level=logging.ERROR)

# Apply nest_asyncio to allow async operations in Jupyter
nest_asyncio.apply()

# Example usage functions
async def scan_binance_btc_current():
    """Scan current candle for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=0)

async def scan_binance_btc_closed():
    """Scan last closed candle for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=1)

async def scan_binance_btc_previous():
    """Scan two candles ago for confluence - Binance BTC pairs"""
    await run_binance_btc_confluence_scanner("1w", offset=2)

# Main execution function
async def main():
    """
    Main execution - modify parameters here
    """
    timeframe = "1w"  # 1d, 2d, 3d, 1w
    offset = 0        # 0 = current candle, 1 = last closed candle, 2 = two candles ago
    
    await run_binance_btc_confluence_scanner(timeframe, offset)

# Run the async main function
print("🔍 BINANCE BTC CONFLUENCE SCANNER")
print("=" * 40)
print("Available timeframes:")
print("• Minutes: 1m, 3m, 5m, 15m, 30m")
print("• Hours: 1h, 2h, 4h, 6h, 8h, 12h") 
print("• Days: 1d, 2d, 3d")
print("• Weeks/Months: 1w, 1M")
print("\nAvailable functions:")
print("• await main() - Run with default settings")
print("• await scan_binance_btc_current() - Scan current candle")
print("• await scan_binance_btc_closed() - Scan last closed candle")
print("• await scan_binance_btc_previous() - Scan two candles ago")
print("• await run_binance_btc_confluence_scanner('timeframe', offset) - Custom scan")
print("\nExamples:")
print("• await run_binance_btc_confluence_scanner('4h', 1)  # 4-hour last closed")
print("• await run_binance_btc_confluence_scanner('15m', 0) # 15-min current")
print("• await run_binance_btc_confluence_scanner('1M', 1)  # Monthly last closed")
print("This scanner uses REAL Binance BTC pair volumes!")

# Uncomment to auto-run:
await main()

In [ ]:
#Direct strategy debug of any pair on any exchange
import asyncio
import sys
import os
import logging
import pandas as pd
import numpy as np
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")
from exchanges import MexcSpotClient, BybitSpotClient, GateioSpotClient, KucoinSpotClient, BinanceSpotClient, BinanceFuturesClient, BybitFuturesClient
from custom_strategies import detect_volume_surge, detect_weak_uptrend, detect_pin_down
from breakout_vsa import vsa_detector, breakout_bar_vsa, stop_bar_vsa, reversal_bar_vsa, start_bar_vsa, loaded_bar_vsa, test_bar_vsa

async def test_strategy(exchange_client_class, timeframe, symbol, strategy_name):
    client = exchange_client_class(timeframe=timeframe)
    await client.init_session()
    df = await client.fetch_klines(symbol)
    await client.close_session()
    
    if df is None or len(df) < 10:
        print(f"No data fetched for {symbol} or insufficient data (< 10 bars)")
        return
    
    print(f"{timeframe} Candles for {symbol}:")
    print(df.tail(5))
    last_row = df.iloc[-1]
    volume_usd = last_row['volume'] * last_row['close']
    print(f"Last Bar: volume_usd={volume_usd:.2f}, close={last_row['close']}, volume={last_row['volume']:.2f}")
    
    # Different handling based on strategy type
    if strategy_name == "volume_surge":
        # Use detect_volume_surge directly
        detected, result = detect_volume_surge(df)
        
        print(f"\nVolume Surge Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nVolume Surge Details:")
            print(f"  Date: {result['timestamp']}")
            print(f"  Close: ${result['close_price']:,.8f}")
            print(f"  Volume: {result['volume']:,.2f}")
            print(f"  Volume USD: ${result['volume_usd']:,.2f}")
            print(f"  Volume Ratio: {result['volume_ratio']:,.2f}x")
            print(f"  Score: {result['score']:,.2f}")
            print(f"  Price Extreme: {result['price_extreme']}")
    
    elif strategy_name == "pin_down":
        from custom_strategies import detect_pin_down
        detected, result = detect_pin_down(df)
        
        print(f"\nPin Down Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nPin Down Details:")
            for key, value in result.items():
                if key != 'symbol':  # Skip symbol as we already know it
                    print(f"  {key}: {value}")
    
    elif strategy_name == "weak_uptrend":
        from custom_strategies import detect_weak_uptrend
        detected, result = detect_weak_uptrend(df)
        
        print(f"\nWeak Uptrend Detection Results:")
        print(f"Detected: {detected}")
        
        if detected:
            print(f"\nWeak Uptrend Details:")
            for key, value in result.items():
                if key != 'symbol':  # Skip symbol as we already know it
                    print(f"  {key}: {value}")
    
    else:
        # For VSA strategies, import the appropriate get_params
        if strategy_name == "reversal_bar":
            from breakout_vsa.strategies.reversal_bar import get_params
        elif strategy_name == "breakout_bar":
            from breakout_vsa.strategies.breakout_bar import get_params
        elif strategy_name == "loaded_bar":
            from breakout_vsa.strategies.loaded_bar import get_params
        elif strategy_name == "stop_bar":
            from breakout_vsa.strategies.stop_bar import get_params
        elif strategy_name == "start_bar":
            from breakout_vsa.strategies.start_bar import get_params
        else:
            print(f"Unknown strategy: {strategy_name}")
            return
        
        # Use vsa_detector with strategy-specific params
        params = get_params()
        condition, result = vsa_detector(df, params)
        
        strategy_display_name = strategy_name.replace('_vsa', '').replace('_', ' ').title()
        print(f"\n{strategy_display_name} Detection Results:")
        print(f"Current Bar (index -1): {condition.iloc[-1]}")
        if len(df) > 1:
            print(f"Last Closed Bar (index -2): {condition.iloc[-2]}")
        
        if condition.iloc[-1] or (len(df) > 1 and condition.iloc[-2]):
            detected_idx = -1 if condition.iloc[-1] else -2
            volume_mean = df['volume'].rolling(7).mean().iloc[detected_idx]
            bar_range = df['high'].iloc[detected_idx] - df['low'].iloc[detected_idx]
            close_off_low = (df['close'].iloc[detected_idx] - df['low'].iloc[detected_idx]) / bar_range * 100 if bar_range > 0 else 0
            volume_usd_detected = df['volume'].iloc[detected_idx] * df['close'].iloc[detected_idx]
            
            arctan_ratio = result['arctan_ratio'].iloc[detected_idx]  # From result DataFrame
            
            print(f"\nDetected at index {detected_idx} ({'Current' if detected_idx == -1 else 'Last Closed'} Bar):")
            print(f"  Date: {df.index[detected_idx]}")
            print(f"  Close: ${df['close'].iloc[detected_idx]:,.8f}")
            print(f"  Volume Ratio: {df['volume'].iloc[detected_idx] / volume_mean if volume_mean > 0 else 0:.2f}x")
            print(f"  {timeframe} Volume: ${volume_usd_detected:.2f}")
            print(f"  Close Off Low: {close_off_low:.1f}%")
            print(f"  Angular Ratio: {arctan_ratio:.2f}")

# Define the test case
exchange_client = GateioSpotClient
timeframe = "1w"
symbol = "PRCL_USDT"
strategy = "loaded_bar"
await test_strategy(exchange_client, timeframe, symbol, strategy)

In [ ]:
#zip the project

import shutil
import os

# Go to parent directory of your project
os.chdir("/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025")

# Create the zip file (this will include everything inside 'hbs_2025')
shutil.make_archive("Project_VSA_2025_backup", 'zip', "Project")


In [ ]:
#!/usr/bin/env python3 -> Debug built weekly candles for mexc and kucoin 
import sys
import os
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)
print(f"✓ Added {project_dir} to sys.path")
import asyncio
import logging
import pandas as pd
import numpy as np
from exchanges.kucoin_client import KucoinClient
from breakout_vsa.core import calculate_start_bar

from scanner.main import kline_cache
kline_cache.clear()  # Clear cache for fresh data

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

async def debug_start_bar_detection():
    # Initialize client
    client = KucoinClient(timeframe="1w")
    await client.init_session()
    
    # Symbol to debug
    symbol = "TAO-USDT"
    
    try:
        # Fetch data
        df = await client.fetch_klines(symbol)
        
        if df is not None:
            print(f"Weekly candles for {symbol}:")
            print(df.tail())
            
            # Add intermediate calculations to see what's happening
            # This is a modified version of calculate_start_bar that adds debugging
            lookback = 5
            volume_lookback = 30
            volume_percentile = 50
            low_percentile = 75
            range_percentile = 75
            close_off_lows_percent = 50
            prev_close_range = 75
            
            # Calculate basic bar characteristics
            df['bar_range'] = df['high'] - df['low']
            df['volume_rank'] = df['volume'].rolling(lookback).apply(
                lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                raw=True
            )
            
            # Calculate rolling values
            df['macro_low'] = df['low'].rolling(volume_lookback).min()
            df['macro_high'] = df['high'].rolling(volume_lookback).max()
            df['highest_high'] = df['high'].rolling(lookback).max()
            
            # Volume conditions
            df['volume_sma'] = df['volume'].rolling(volume_lookback).mean()
            df['volume_std'] = df['volume'].rolling(volume_lookback).std()
            df['excess_volume'] = df['volume'] > (df['volume_sma'] + 3.0 * df['volume_std'])
            
            # Range conditions
            df['range_sma'] = df['bar_range'].rolling(volume_lookback).mean()
            df['range_std'] = df['bar_range'].rolling(volume_lookback).std()
            df['excess_range'] = df['bar_range'] > (df['range_sma'] + 3.0 * df['range_std'])
            
            # Volume percentile condition
            def is_in_top_percent(series, length, percent):
                ranks = series.rolling(length).apply(
                    lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                    raw=True
                )
                return ranks >= percent
            
            def is_in_bottom_percent(series, length, percent):
                ranks = series.rolling(length).apply(
                    lambda x: sum(1.0 for val in x if val <= x[-1]) / len(x) * 100, 
                    raw=True
                )
                return ranks <= percent
            
            # Volume conditions
            df['is_higher_volume'] = is_in_top_percent(df['volume'], lookback, volume_percentile)
            df['is_high_volume'] = (df['volume'] > 0.75 * df['volume_sma']) & (df['volume'] > df['volume'].shift(1))
            
            # Price action conditions
            df['has_higher_high'] = df['high'] > df['high'].shift(1)
            df['no_narrow_range'] = is_in_top_percent(df['bar_range'], lookback, range_percentile)
            
            # Low price condition
            df['is_in_the_lows'] = (
                (df['low'] - df['macro_low']).abs() < df['bar_range']
            ) | is_in_bottom_percent(df['low'], volume_lookback, low_percentile)
            
            # Close position conditions
            df['close_in_the_highs'] = (
                (df['close'] - df['low']) / df['bar_range']
            ) >= (close_off_lows_percent / 100)
            
            # Previous close distance condition
            df['far_prev_close'] = (
                (df['close'] - df['close'].shift(1)).abs() >=
                (df['bar_range'].shift(1) * (prev_close_range / 100))
            )
            
            # New highs condition
            df['new_highs'] = df['high'] >= 0.75 * df['highest_high']
            
            # Optional strength condition
            df['strong_close'] = df['close'] >= df['highest_high'].shift(1)
            
            # Now check the actual values for the last few bars
            last_rows = df.tail(3)
            
            print("\nAnalyzing last 3 bars:")
            for idx, row in last_rows.iterrows():
                print(f"\nBar at {idx.strftime('%Y-%m-%d')}:")
                print(f"  is_high_volume: {row['is_high_volume']}")
                print(f"  has_higher_high: {row['has_higher_high']}")
                print(f"  no_narrow_range: {row['no_narrow_range']}")
                print(f"  close_in_the_highs: {row['close_in_the_highs']}")
                print(f"  far_prev_close: {row['far_prev_close']}")
                print(f"  excess_range: {row['excess_range']}")
                print(f"  excess_volume: {row['excess_volume']}")
                print(f"  new_highs: {row['new_highs']}")
                print(f"  is_in_the_lows: {row['is_in_the_lows']}")
                print(f"  volume: {row['volume']}, volume_sma: {row['volume_sma']}")
                print(f"  bar_range: {row['bar_range']}, range_sma: {row['range_sma']}")
                
            # Run the original function to confirm
            start_bar_pattern = calculate_start_bar(df)
            print(f"\nFinal Start Bar detection result:")
            print(start_bar_pattern.tail(3))
            
    except Exception as e:
        print(f"Error in debug: {str(e)}")
    finally:
        await client.close_session()

# Replace the last part of your script with this:
if __name__ == "__main__":
    try:
        # For Jupyter/IPython environments
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(debug_start_bar_detection())
    except ImportError:
        # For regular Python environments
        asyncio.run(debug_start_bar_detection())

In [ ]:
#zip the project

import shutil
import os

# Go to parent directory of your project
os.chdir("/home/jovyan/work/Crypto/sevenfigures-bot/hbs_2025")

# Create the zip file (this will include everything inside 'hbs_2025')
shutil.make_archive("Project_VSA_2025_backup", 'zip', "Project")

In [ ]:
# ARCHIVE - Confluence Scanner with bar offset

from telegram.ext import Application
import logging
import pandas as pd
import numpy as np
import asyncio
import logging
import nest_asyncio
from datetime import datetime
from tqdm.asyncio import tqdm
import sys
import os
import html

# Add project path
project_dir = os.path.join(os.getcwd(), "Project")
sys.path.insert(0, project_dir)

from exchanges.sf_pairs_service import SFPairsService
from custom_strategies import detect_confluence

class ConfluenceScanner:
    def __init__(self, telegram_token, telegram_chat_id, exchange, timeframe, offset=1):
        self.telegram_token = telegram_token
        self.telegram_chat_id = telegram_chat_id
        self.telegram_app = None
        self.exchange = exchange
        self.timeframe = timeframe
        self.offset = offset  # Added offset parameter
        self.sf_service = SFPairsService()
        
    async def init_telegram(self):
        if self.telegram_app is None:
            self.telegram_app = Application.builder().token(self.telegram_token).build()

    async def send_telegram_alert(self, results):
        if not results:
            return
            
        try:
            message = f"🚨 Confluence Detection - {self.exchange} {self.timeframe}\n\n"
            
            # Map timeframe to TradingView format
            tv_timeframe_map = {
                "1d": "1D",
                "2d": "2D",
                "1w": "1W"
            }
            tv_timeframe = tv_timeframe_map.get(self.timeframe.lower(), self.timeframe)
            
            for result in results:
                exchange_name = self.exchange.upper()
                formatted_symbol = f"{result['symbol']}"
                tv_link = f"https://www.tradingview.com/chart/?symbol={exchange_name}:{formatted_symbol}&interval={tv_timeframe}"
                
                # Escape HTML entities in the URL
                escaped_link = html.escape(tv_link)
                
                # Format according to specified requirements
                time_str = ""
                if result.get('timestamp') is not None:
                    time_str = f"Time: {result['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\n"
                
                message += (
                    f"Symbol: {result['symbol']}\n"
                    f"{time_str}"
                    f"Volume USD: ${result['volume_usd']:,.2f}\n"
                    f"Close: <a href='{escaped_link}'>${result['close']:,.8f}</a>\n"
                    f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                    f"Close Off Low: {result['close_off_low']:.1f}%\n"
                    f"{'='*30}\n"
                )
            
            # Split message more carefully to avoid breaking HTML tags
            max_length = 4000  # Reduced from 4096 to be safer
            
            if len(message) > max_length:
                # Split at natural breaks (between results) to avoid breaking HTML
                sections = message.split('='*30 + '\n')
                current_chunk = ""
                
                for section in sections:
                    if len(current_chunk + section + '='*30 + '\n') > max_length:
                        if current_chunk:
                            await self.telegram_app.bot.send_message(
                                chat_id=self.telegram_chat_id,
                                text=current_chunk.strip(),
                                parse_mode='HTML',
                                disable_web_page_preview=True
                            )
                        current_chunk = section + '\n'
                    else:
                        current_chunk += section + '='*30 + '\n'
                
                # Send remaining chunk
                if current_chunk.strip():
                    await self.telegram_app.bot.send_message(
                        chat_id=self.telegram_chat_id,
                        text=current_chunk.strip(),
                        parse_mode='HTML',
                        disable_web_page_preview=True
                    )
            else:
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=message,
                    parse_mode='HTML',
                    disable_web_page_preview=True
                )
                
        except Exception as e:
            logging.error(f"Error sending Telegram alert: {str(e)}")
            
            # Fallback: send without HTML formatting
            try:
                simple_message = f"🚨 Confluence Detection - {self.exchange} {self.timeframe}\n\n"
                for result in results:
                    simple_message += (
                        f"Symbol: {result['symbol']}\n"
                        f"Volume USD: ${result['volume_usd']:,.2f}\n"
                        f"Close: ${result['close']:,.8f}\n"
                        f"Volume Ratio: {result['volume_ratio']:.2f}x\n"
                        f"Components: Vol={result['high_volume']}, Spread={result['spread_breakout']}, Mom={result['momentum_breakout']}\n\n"
                    )
                
                await self.telegram_app.bot.send_message(
                    chat_id=self.telegram_chat_id,
                    text=simple_message,
                    disable_web_page_preview=True
                )
            except Exception as fallback_error:
                logging.error(f"Fallback Telegram send also failed: {str(fallback_error)}")

    def prepare_sf_data(self, raw_df):
        """Convert SF data to confluence-compatible format"""
        if raw_df is None or len(raw_df) == 0:
            return None
        
        df = pd.DataFrame(raw_df)
        
        # Convert datetime column to pandas datetime and set as index
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime')
        elif 'time' in df.columns:
            # Convert Unix timestamp to datetime
            df['time'] = pd.to_datetime(df['time'], unit='ms')
            df = df.set_index('time')
        
        # Select only OHLCV columns needed for confluence
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        available_cols = [col for col in required_cols if col in df.columns]
        
        if len(available_cols) != 5:
            return None
        
        # Select and clean data
        result_df = df[required_cols].copy()
        
        # Ensure numeric types
        for col in required_cols:
            result_df[col] = pd.to_numeric(result_df[col], errors='coerce')
        
        # Drop any NaN rows
        result_df = result_df.dropna()
        
        return result_df

    def scan_single_market(self, pair, ohlcv_data):
        """Scan a single market for Confluence pattern in the specified bar"""
        try:
            # Prepare data for confluence analysis
            df = self.prepare_sf_data(ohlcv_data)
            
            if df is None or len(df) < 50:  # Need enough data for confluence
                return None
            
            # Calculate which bar to check based on offset
            check_bar = -(self.offset + 1)  # offset=0 means current bar (-1), offset=1 means last closed (-2), etc.
            
            # Run confluence detection
            detected, result = detect_confluence(df, check_bar=check_bar)
            
            if detected:
                # Calculate volume in USD for the target bar
                target_close = df['close'].iloc[check_bar]
                target_volume = df['volume'].iloc[check_bar]
                volume_usd = float(target_close) * float(target_volume)
                
                confluence_result = {
                    'symbol': f"{pair['Token']}{pair['Quote']}",
                    'volume_usd': volume_usd,
                    'close': float(target_close),
                    'volume': float(target_volume),
                    'volume_ratio': result['volume_ratio'],
                    'close_off_low': result['close_off_low'],
                    'momentum_score': result['momentum_score'],
                    'high_volume': result['high_volume'],
                    'spread_breakout': result['spread_breakout'],
                    'momentum_breakout': result['momentum_breakout'],
                    'bar_range': result['bar_range']
                }
                return confluence_result
                
        except Exception as e:
            logging.error(f"Error processing {pair['Token']}{pair['Quote']}: {str(e)}")
        return None

    async def scan_all_markets(self):
        """Scan all markets for Confluence pattern"""
        await self.init_telegram()
        try:
            # Define volume thresholds
            volume_thresholds = {
                "1w": 300000,
                "2d": 100000,
                "1d": 50000
            }
            min_volume = volume_thresholds.get(self.timeframe.lower(), 50000)
            
            # Create offset description
            if self.offset == 0:
                offset_desc = "current candle"
            elif self.offset == 1:
                offset_desc = "last closed candle"
            else:
                offset_desc = f"{self.offset} candles ago"
            
            print(f"Scanning for Confluence patterns in {offset_desc}...")
            print(f"Minimum volume threshold: ${min_volume:,.0f}")
            
            # Get all pairs from SF service
            pairs = self.sf_service.get_pairs_of_exchange(self.exchange)
            print(f"Found {len(pairs)} markets to scan...")
            
            # Process all pairs with progress bar
            all_results = []
            with tqdm(total=len(pairs), desc="Scanning markets") as pbar:
                for pair in pairs:
                    try:
                        # Get OHLCV data from SF service
                        ohlcv_data = self.sf_service.get_ohlcv_for_pair(
                            pair['Token'], 
                            pair['Quote'], 
                            self.exchange, 
                            self.timeframe, 
                            100  # Get more data for confluence analysis
                        )
                        
                        if ohlcv_data is None or len(ohlcv_data) == 0:
                            pbar.update(1)
                            continue
                        
                        df = pd.DataFrame(ohlcv_data)
                        
                        # Check if we have enough data
                        if len(df) >= 50:  # Need enough for confluence analysis
                            target_idx = -(self.offset + 1)  # Adjust index based on offset
                            
                            # Check volume threshold for the target candle
                            try:
                                target_candle_volume = float(df['close'].iloc[target_idx]) * float(df['volume'].iloc[target_idx])
                                
                                # Only process if volume meets threshold
                                if target_candle_volume >= min_volume:
                                    result = self.scan_single_market(pair, ohlcv_data)
                                    if result:
                                        all_results.append(result)
                                        print(f"Found Confluence: {pair['Token']}{pair['Quote']} 🎯")
                            except (IndexError, ValueError):
                                pass  # Skip if we can't calculate volume
                                    
                    except Exception as e:
                        if "500" not in str(e):  # Don't log 500 errors
                            logging.error(f"Error processing {pair['Token']}{pair['Quote']}: {str(e)}")
                    finally:
                        pbar.update(1)
            
            # Sort by volume
            all_results.sort(key=lambda x: x['volume_usd'], reverse=True)
            
            # Send Telegram alert if we found any patterns
            if all_results:
                await self.send_telegram_alert(all_results)
            
            return all_results
            
        except Exception as e:
            logging.error(f"Error scanning markets: {str(e)}")
            return []

async def run_confluence_scanner(exchange, timeframe, offset=1):
    """
    Run the Confluence scanner
    
    Parameters:
    exchange (str): Exchange name (Kucoin, Mexc, Binance)
    timeframe (str): Time period (1d, 2d, 1w)
    offset (int): Bar offset (0=current, 1=last closed, 2=two bars ago, etc.)
    """
    
    if offset == 0:
        offset_desc = "current candle"
    elif offset == 1:
        offset_desc = "last closed candle"
    else:
        offset_desc = f"{offset} candles ago"
    
    print(f"Starting Confluence scan for {offset_desc} on {exchange} {timeframe}...")
    
    # Use the confluence telegram token from your big project config
    # You should replace this with the actual token from utils/config.py TELEGRAM_TOKENS["confluence"]
    telegram_token = "8066329517:AAHVr6kufZWe8UqCKPfmsRhSPleNlt_7G-g"  # Replace with confluence token
    telegram_chat_id = "375812423"  # Your chat ID
    
    scanner = ConfluenceScanner(telegram_token, telegram_chat_id, exchange, timeframe, offset)
    results = await scanner.scan_all_markets()
    
    if results:
        print(f"\nFound {len(results)} Confluence patterns:")
        
        # Convert results to DataFrame for console display
        df_results = pd.DataFrame(results)
        
        # Round numeric columns
        df_results['volume_usd'] = df_results['volume_usd'].round(2)
        df_results['close'] = df_results['close'].round(8)
        df_results['volume'] = df_results['volume'].round(2)
        df_results['volume_ratio'] = df_results['volume_ratio'].round(2)
        df_results['close_off_low'] = df_results['close_off_low'].round(1)
        df_results['momentum_score'] = df_results['momentum_score'].round(4)
        
        # Reorder columns for better display
        display_cols = ['symbol', 'close', 'volume_usd', 'volume_ratio', 'close_off_low', 
                       'momentum_score', 'high_volume', 'spread_breakout', 'momentum_breakout']
        available_cols = [col for col in display_cols if col in df_results.columns]
        
        # Display the results
        print(df_results[available_cols])
        
        # Show component analysis
        print(f"\n🔧 COMPONENT ANALYSIS:")
        vol_count = df_results['high_volume'].sum()
        spread_count = df_results['spread_breakout'].sum()
        momentum_count = df_results['momentum_breakout'].sum()
        
        print(f"High Volume signals: {vol_count}/{len(results)} ({vol_count/len(results)*100:.1f}%)")
        print(f"Spread Breakout signals: {spread_count}/{len(results)} ({spread_count/len(results)*100:.1f}%)")
        print(f"Momentum Breakout signals: {momentum_count}/{len(results)} ({momentum_count/len(results)*100:.1f}%)")
        
    else:
        print(f"\nNo Confluence patterns found in {offset_desc}")

# Set up logging
logging.basicConfig(level=logging.ERROR)

# Apply nest_asyncio to allow async operations in Jupyter
nest_asyncio.apply()

# Example usage functions
async def scan_current_confluence():
    """Scan current candle for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=0)

async def scan_closed_confluence():
    """Scan last closed candle for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=1)

async def scan_previous_confluence():
    """Scan two candles ago for confluence"""
    await run_confluence_scanner("Kucoin", "1w", offset=2)

# Main execution function
async def main():
    """
    Main execution - modify parameters here
    """
    exchange = "Mexc"  # Binance, Kucoin, Mexc
    timeframe = "1w"     # 1d, 2d, 1w
    offset = 0           # 0 = current candle, 1 = last closed candle, 2 = two candles ago
    
    await run_confluence_scanner(exchange, timeframe, offset)

# Run the async main function
print("🔍 CONFLUENCE SCANNER")
print("=" * 30)
print("Available functions:")
print("• await main() - Run with default settings")
print("• await scan_current_confluence() - Scan current candle")
print("• await scan_closed_confluence() - Scan last closed candle")
print("• await scan_previous_confluence() - Scan two candles ago")
print("• await run_confluence_scanner('Exchange', 'timeframe', offset) - Custom scan")
print("\nExample: await main()")

# Uncomment to auto-run:
await main()

In [ ]:
# Standalone HLC Bar Chart Plotter
"""
Standalone HLC Bar Chart Plotter
A reusable function for plotting HLC bars with optional pattern highlighting
"""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from matplotlib.lines import Line2D

def plot_hlc_bars(data, highlighted_bars=None, title="HLC Chart", symbol="SYMBOL", 
                  interval="1d", semilog=False, highlight_color="fuchsia", 
                  highlight_label="Pattern", figsize=(14, 10), show_volume=True):
    """
    Plot HLC bar chart with optional pattern highlighting
    
    Parameters:
    -----------
    data : pandas.DataFrame
        DataFrame with columns: ['datetime', 'high', 'low', 'close', 'volume']
        - datetime: timestamp column (will be used for x-axis)
        - high: high prices
        - low: low prices  
        - close: close prices
        - volume: volume data (optional if show_volume=False)
        
    highlighted_bars : pandas.Series or list/array, optional
        Boolean series or array indicating which bars to highlight
        Length must match data length
        
    title : str, default "HLC Chart"
        Chart title
        
    symbol : str, default "SYMBOL" 
        Symbol name for display
        
    interval : str, default "1d"
        Time interval for date formatting (1m, 5m, 15m, 30m, 1h, 4h, 1d, 3d, 1w, 1M)
        
    semilog : bool, default False
        Use logarithmic scale for price chart
        
    highlight_color : str, default "fuchsia"
        Color for highlighted bars
        
    highlight_label : str, default "Pattern"
        Label for highlighted bars in legend
        
    figsize : tuple, default (14, 10)
        Figure size (width, height)
        
    show_volume : bool, default True
        Whether to show volume subplot
        
    Returns:
    --------
    matplotlib.figure.Figure
        The created figure object
    """
    
    # Input validation
    if not isinstance(data, pd.DataFrame):
        raise ValueError("data must be a pandas DataFrame")
    
    # Check required columns
    required_cols = ['datetime', 'high', 'low', 'close']
    if show_volume:
        required_cols.append('volume')
    
    missing_cols = [col for col in required_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    if len(data) == 0:
        raise ValueError("data DataFrame is empty")
    
    # Validate highlighted_bars
    if highlighted_bars is not None:
        if len(highlighted_bars) != len(data):
            raise ValueError("highlighted_bars length must match data length")
        # Convert to boolean array
        highlighted_bars = np.array(highlighted_bars, dtype=bool)
    else:
        highlighted_bars = np.zeros(len(data), dtype=bool)
    
    # Create figure with subplots
    if show_volume:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, 
                                       gridspec_kw={'height_ratios': [3, 1]})
    else:
        fig, ax1 = plt.subplots(1, 1, figsize=figsize)
        ax2 = None
    
    # Convert dates to numbers for plotting
    dates = mdates.date2num(data['datetime'])
    
    # Calculate margins and tick length
    date_range = dates[-1] - dates[0] if len(dates) > 1 else 1
    margin = date_range * 0.05
    
    # Calculate actual bar spacing for consistent tick length
    if len(dates) > 1:
        avg_bar_spacing = date_range / (len(dates) - 1)
        tick_length = avg_bar_spacing * 0.4  # 40% of bar spacing
        volume_bar_width = avg_bar_spacing * 0.8
    else:
        tick_length = date_range * 0.01
        volume_bar_width = date_range * 0.02
    
    # Draw HLC bars
    for i, (date, high, low, close) in enumerate(zip(dates, data['high'], data['low'], data['close'])):
        is_highlighted = highlighted_bars[i]
        color = highlight_color if is_highlighted else 'black'
        line_width = 1.2
        
        # Vertical line from low to high
        ax1.plot([date, date], [low, high], color=color, linewidth=line_width, solid_capstyle='butt')
        
        # Horizontal tick mark for close (on the right side)
        ax1.plot([date, date + tick_length], [close, close], color=color, 
                linewidth=line_width+0.5, solid_capstyle='butt')
    
    # Configure price chart
    if semilog:
        ax1.set_yscale('log')
        scale_text = "Semilog Scale"
    else:
        scale_text = "Linear Scale"
    
    ax1.set_title(f'{symbol} {title} - {scale_text}', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Price', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Format x-axis based on timeframe
    _format_datetime_axis(ax1, interval)
    ax1.set_xlim(dates[0] - margin, dates[-1] + margin)
    
    # Volume chart
    if show_volume and ax2 is not None:
        volume_colors = [highlight_color if highlighted_bars[i] else 'orange' for i in range(len(dates))]
        volume_edges = ['darkmagenta' if highlighted_bars[i] else 'darkorange' for i in range(len(dates))]
        
        ax2.bar(dates, data['volume'], width=volume_bar_width, alpha=0.6, 
                color=volume_colors, edgecolor=volume_edges)
        ax2.set_ylabel('Volume', fontsize=12)
        ax2.set_xlabel('Date', fontsize=12)
        ax2.grid(True, alpha=0.3)
        
        _format_datetime_axis(ax2, interval)
        ax2.set_xlim(dates[0] - margin, dates[-1] + margin)
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    else:
        ax1.set_xlabel('Date', fontsize=12)
    
    # Create legend
    legend_elements = [
        Line2D([0], [0], color='black', linewidth=2, label='High-Low Range'),
        Line2D([0], [0], color='black', linewidth=3, label='Close Price (right tick)')
    ]
    
    # Add highlighted bars to legend if any exist
    if highlighted_bars.any():
        legend_elements.append(
            Line2D([0], [0], color=highlight_color, linewidth=3, label=highlight_label)
        )
        highlight_count = highlighted_bars.sum()
    else:
        highlight_count = 0
    
    ax1.legend(handles=legend_elements, loc='upper left', title=scale_text)
    
    # Add pattern count if patterns exist
    if highlight_count > 0:
        ax1.text(0.99, 0.95, f'{highlight_label}: {highlight_count}', 
                transform=ax1.transAxes, ha='right', va='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                fontsize=10)
    
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    plt.tight_layout()
    
    return fig

def _format_datetime_axis(ax, interval):
    """Helper function to format datetime axis based on interval"""
    if interval in ['1m', '5m', '15m', '30m']:
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
    elif interval in ['1h', '2h', '4h', '6h', '12h']:
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    elif interval in ['1d', '3d']:
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    else:  # weekly, monthly
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Example usage and test function
def example_usage():
    """Example showing how to use the plot_hlc_bars function"""
    
    # Create sample data
    import datetime
    dates = pd.date_range('2023-01-01', periods=100, freq='D')
    np.random.seed(42)
    
    # Generate realistic OHLC data
    closes = 100 + np.cumsum(np.random.randn(100) * 0.02)
    highs = closes + np.random.rand(100) * 5
    lows = closes - np.random.rand(100) * 5
    volumes = np.random.rand(100) * 1000000
    
    data = pd.DataFrame({
        'datetime': dates,
        'high': highs,
        'low': lows, 
        'close': closes,
        'volume': volumes
    })
    
    # Create some random pattern detections
    pattern_detected = np.random.choice([True, False], size=100, p=[0.1, 0.9])
    
    # Plot with pattern highlighting
    fig = plot_hlc_bars(
        data=data,
        highlighted_bars=pattern_detected,
        title="Daily Chart with Pattern Detection",
        symbol="EXAMPLE",
        interval="1d",
        semilog=False,
        highlight_color="red",
        highlight_label="Detected Pattern"
    )
    
    plt.show()
    return fig

# Scanner integration example
def scanner_integration_example():
    """Example of how to integrate with a scanner function"""
    
    def my_pattern_scanner(data):
        """
        Example scanner function - replace with your actual scanner logic
        Returns boolean array indicating pattern detection
        """
        # Example: detect when close > 20-period moving average
        ma20 = data['close'].rolling(20).mean()
        pattern = (data['close'] > ma20) & (data['volume'] > data['volume'].rolling(10).mean())
        return pattern.fillna(False)
    
    # Your data loading logic here
    # data = load_your_data()  # Replace with actual data loading
    
    # For demo, create sample data
    dates = pd.date_range('2023-01-01', periods=200, freq='D')
    np.random.seed(42)
    closes = 100 + np.cumsum(np.random.randn(200) * 0.02)
    highs = closes + np.random.rand(200) * 3
    lows = closes - np.random.rand(200) * 3
    volumes = np.random.rand(200) * 1000000
    
    data = pd.DataFrame({
        'datetime': dates,
        'high': highs,
        'low': lows,
        'close': closes,
        'volume': volumes
    })
    
    # Run your scanner
    detected_patterns = my_pattern_scanner(data)
    
    # Plot only if patterns are detected
    if detected_patterns.any():
        print(f"Patterns detected! Found {detected_patterns.sum()} occurrences")
        fig = plot_hlc_bars(
            data=data,
            highlighted_bars=detected_patterns,
            title="Scanner Results",
            symbol="SCANNED_SYMBOL",
            interval="1d",
            highlight_color="lime",
            highlight_label="Scanner Hit"
        )
        plt.show()
    else:
        print("No patterns detected")

if __name__ == "__main__":
    # Run example
    example_usage()

In [ ]:
#Dashboard
from pyngrok import ngrok
import os, sys, subprocess, time

# --- ngrok auth (DO NOT commit your token) ---
ngrok.set_auth_token("31mFDQNYuBuJw7mTKNxyDZLbZag_4Q1mV2EEggMBGYecRRZyF")

# Clean old tunnels
try:
    ngrok.kill()
except Exception:
    pass

# Pick a port
port = 8501

# Launch Streamlit dashboard in the background
# Ensure working directory contains dashboard.py
cmd = ["streamlit", "run", "dashboard.py", "--server.address", "0.0.0.0", "--server.port", str(port)]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Give Streamlit a moment to boot
time.sleep(3)

# Create public tunnel
public_url = ngrok.connect(addr=port)
print("Streamlit is running! Access it at:", public_url)


In [ ]:
# Simple Historical Signal Scanner - Scan a specific pair or all pairs for historical signals

import asyncio
import nest_asyncio
import pandas as pd
import warnings
import logging

# Enable nested event loops for Jupyter
nest_asyncio.apply()
warnings.filterwarnings('ignore')

# Import our scanner module
from direct_test import (
    HistoricalSignalScanner, 
    save_signals_to_csv,
    analyze_signals,
    filter_signals
)

print("Historical Signal Scanner Ready!")
print("=" * 50)

# =============================================================================
# MAIN SCANNING FUNCTION
# =============================================================================

async def scan_historical_signals(symbol=None, exchange="binance_futures", timeframe="1d", 
                                 strategies=None, max_symbols=None):
    """
    Scan for historical signals
    
    Args:
        symbol: Specific symbol to scan (e.g., "BTCUSDT") or None to scan all
        exchange: Exchange to use (default: "binance_futures")
        timeframe: Timeframe to analyze (default: "1d")
        strategies: List of strategies or None for all available
        max_symbols: Max number of symbols to scan when symbol=None (default: 50)
    
    Returns:
        pandas.DataFrame: All detected signals
    """
    
    # Default strategies if none provided
    if strategies is None:
        strategies = ['hbs_breakout', 'confluence', 'wedge_breakout', 'channel_breakout', 
                     'consolidation_breakout', 'sma50_breakout']
    
    # Default max symbols
    if max_symbols is None:
        max_symbols = 50
    
    scanner = HistoricalSignalScanner(exchange, timeframe)
    
    try:
        await scanner.initialize()
        print(f"Initialized {exchange} scanner for {timeframe} timeframe")
        
        if symbol:
            # Scan specific symbol
            print(f"Scanning {symbol}...")
            results = await scanner.scan_symbol_historical(symbol, strategies)
            df = scanner.create_signals_dataframe(results)
            
            if not df.empty:
                filename = f"{symbol}_{exchange}_{timeframe}_signals"
                save_signals_to_csv(df, filename)
                print(f"\nFound {len(df)} signals for {symbol}")
                analyze_signals(df)
            else:
                print(f"No signals found for {symbol}")
                
        else:
            # Scan all symbols
            print(f"Getting symbols from {exchange}...")
            symbols = await scanner.get_all_symbols(limit=max_symbols)
            print(f"Scanning {len(symbols)} symbols...")
            
            results = await scanner.scan_multiple_symbols(symbols, strategies)
            df = scanner.create_signals_dataframe(results)
            
            if not df.empty:
                filename = f"all_symbols_{exchange}_{timeframe}_{len(symbols)}pairs_signals"
                save_signals_to_csv(df, filename)
                print(f"\nFound {len(df)} total signals across {len(symbols)} symbols")
                analyze_signals(df)
                
                # Show top symbols by signal count
                print(f"\nTop symbols by signal count:")
                symbol_counts = df['symbol'].value_counts().head(10)
                for sym, count in symbol_counts.items():
                    print(f"  {sym}: {count} signals")
            else:
                print(f"No signals found across {len(symbols)} symbols")
        
        return df
        
    finally:
        await scanner.close()

# =============================================================================
# QUICK COMMANDS
# =============================================================================

async def scan_pair(symbol, exchange="binance_futures", timeframe="1d"):
    """Quick scan for a specific trading pair"""
    return await scan_historical_signals(symbol=symbol, exchange=exchange, timeframe=timeframe)

async def scan_all_pairs(exchange="binance_futures", timeframe="1d", max_pairs=50):
    """Quick scan for all trading pairs"""
    return await scan_historical_signals(symbol=None, exchange=exchange, 
                                        timeframe=timeframe, max_symbols=max_pairs)


df = await scan_historical_signals(exchange="mexc_spot", timeframe="2d",
    symbol="NOS_USDT",
    strategies=['hbs_breakout']
)

In [ ]:
#ASCII CHART + Testing mexc 1w candles

import requests
import datetime as dt

MEXC_URL = "https://api.mexc.com/api/v3/klines"
SYMBOL = "STBLUSDT"
INTERVAL = "1W"         # MEXC uses uppercase W for weekly klines
LIMIT = 50              # ask for more than 5 so we can always slice down

HEADERS = {"User-Agent": "console-fetch/1.0 (+https://example.com)"}

def fetch_mexc_weekly(symbol=SYMBOL, limit=LIMIT):
    params = {
        "symbol": symbol,
        "interval": INTERVAL,
        "limit": limit
    }
    r = requests.get(MEXC_URL, params=params, headers=HEADERS, timeout=20)
    r.raise_for_status()
    data = r.json()
    if not isinstance(data, list) or not data:
        raise RuntimeError(f"Unexpected response: {data}")

    # Each kline: [openTime, open, high, low, close, volume, closeTime, ...]
    candles = []
    for row in data:
        open_time_ms = int(row[0])
        candles.append({
            "time": dt.datetime.utcfromtimestamp(open_time_ms / 1000.0),
            "open": float(row[1]),
            "high": float(row[2]),
            "low":  float(row[3]),
            "close": float(row[4]),
            "volume": float(row[5]),
        })
    return candles

def ascii_candles(candles, width=40):
    """
    Render simple ASCII candles for a small set of bars.
    For each candle, draw a range [low..high] as a line, and mark:
      O = open, C = close. Up candles also show '+' at close; down show '-'.
    The scale is shared across the provided candles.
    """
    if not candles:
        return

    lo = min(c["low"] for c in candles)
    hi = max(c["high"] for c in candles)
    if hi == lo:
        hi = lo + 1e-9  # avoid zero range

    def pos(price):
        # map a price into [0..width-1]
        return int(round((price - lo) / (hi - lo) * (width - 1)))

    lines = []
    for c in candles:
        line = [" "] * width
        low_p, high_p = pos(c["low"]), pos(c["high"])
        open_p, close_p = pos(c["open"]), pos(c["close"])
        # draw range
        for i in range(low_p, high_p + 1):
            line[i] = "─"
        # mark open/close
        line[open_p] = "O"
        if close_p == open_p:
            # nudge to show both if equal
            close_p = min(width - 1, close_p + 1)
        line[close_p] = "C"
        # up/down marker at close
        line[close_p] = "+" if c["close"] >= c["open"] else "-"
        lines.append("".join(line))
    return lines, lo, hi

def fmt_num(x):
    # compact formatting for prices
    if x >= 1000:
        return f"{x:,.0f}"
    return f"{x:,.2f}"

def main():
    candles = fetch_mexc_weekly()
    last14 = candles[-14:]  # last 5 weekly bars

    # Print a small info table
    print("Last 5 weekly candles (MEXC spot, BTCUSDT, 1W):")
    print("Date (UTC)     |     Open       High        Low       Close        Vol")
    print("-" * 74)
    for c in last14:
        print(f"{c['time'].date()} | {fmt_num(c['open']).rjust(10)}  {fmt_num(c['high']).rjust(10)}  "
              f"{fmt_num(c['low']).rjust(10)}  {fmt_num(c['close']).rjust(10)}  {fmt_num(c['volume']).rjust(10)}")

    print("\nASCII chart (shared scale across the 14 bars):")
    chart, lo, hi = ascii_candles(last14, width=48)
    # Put labels and lines together
    for i, (c, line) in enumerate(zip(last14, chart)):
        label = c["time"].strftime("%Y-%m-%d")
        print(f"{label} | {line} | O={fmt_num(c['open'])} C={fmt_num(c['close'])}")

    print(f"\nScale: low={fmt_num(lo)}  high={fmt_num(hi)}")

if __name__ == "__main__":
    main()


In [ ]:
# Backtest a strategy
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
VS Wakeup Strategy Backtest - Binance Spot (Multiple Pairs)
===========================================================

Backtests the VS Wakeup composed strategy using actual project implementations
on Binance spot data for the last 1000 bars across multiple trading pairs.

Analyzes:
- Highest price after signal (pivot high with 3-bar lookback)
- Maximum drawdown before reaching highest price
- Quick reversal analysis (max high in next 3 bars)
- Performance metrics across all pairs

Author: Market Scanner v2.10
Date: 2024
"""

import pandas as pd
import numpy as np
import aiohttp
import asyncio
import logging
import sys
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add project path to import custom strategies
project_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_dir)

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("VS Wakeup Strategy Backtest - Binance Spot (Multi-Pair)")
print("=" * 60)

# ═══════════════════════════════════════════════════════════════════════════════
# Import Custom Strategies from Project
# ═══════════════════════════════════════════════════════════════════════════════

try:
    from custom_strategies import detect_consolidation, detect_confluence
    print("✓ Successfully imported custom strategies from project")
except ImportError as e:
    print(f"✗ Failed to import custom strategies: {e}")
    print("Please ensure you're running this notebook from the project directory")
    print("or adjust the project_dir path above")
    sys.exit(1)

# ═══════════════════════════════════════════════════════════════════════════════
# Configuration
# ═══════════════════════════════════════════════════════════════════════════════

# Backtest Configuration
TIMEFRAME = '1d'  # Binance supports: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
BARS_TO_FETCH = 1000  # Maximum bars to fetch for backtest
LOOKBACK_BARS = 3  # Bars to look back for pivot high detection
QUICK_REVERSAL_BARS = 3  # Bars to check for quick reversal after signal

# Symbol Selection Options
SYMBOL_MODE = 'top_volume'  # Options: 'top_volume', 'selected', 'all_major'

# Pre-selected high-quality pairs (if using 'selected' mode)
SELECTED_SYMBOLS = [
    'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'ADAUSDT', 'SOLUSDT',
    'XRPUSDT', 'DOTUSDT', 'LINKUSDT', 'LTCUSDT', 'MATICUSDT',
    'AVAXUSDT', 'ATOMUSDT', 'NEARUSDT', 'FTMUSDT', 'SANDUSDT'
]

# Major pairs for comprehensive testing (if using 'all_major' mode)  
MAJOR_SYMBOLS = [
    'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'ADAUSDT', 'SOLUSDT', 'XRPUSDT', 'DOTUSDT',
    'LINKUSDT', 'LTCUSDT', 'MATICUSDT', 'AVAXUSDT', 'ATOMUSDT', 'NEARUSDT', 
    'FTMUSDT', 'SANDUSDT', 'MANAUSDT', 'ALGOUSDT', 'VETUSDT', 'ICPUSDT', 'THETAUSDT',
    'FILUSDT', 'TRXUSDT', 'EOSUSDT', 'XLMUSDT', 'AAVEUSDT', 'MKRUSDT', 'COMPUSDT',
    'SUSHIUSDT', 'YFIUSDT', 'SNXUSDT', 'CRVUSDT', 'UNIUSDT', 'RENUSDT', 'ENJUSDT',
    'CHZUSDT', 'BATUSDT', 'ZRXUSDT', 'STORJUSDT', 'OCEAUSDT', 'SKLUSDT'
]

print(f"Configuration:")
print(f"  Timeframe: {TIMEFRAME}")
print(f"  Bars to fetch: {BARS_TO_FETCH}")
print(f"  Symbol mode: {SYMBOL_MODE}")
print(f"  Pivot lookback: {LOOKBACK_BARS} bars")
print(f"  Quick reversal check: {QUICK_REVERSAL_BARS} bars")

# ═══════════════════════════════════════════════════════════════════════════════
# Binance Data Client
# ═══════════════════════════════════════════════════════════════════════════════

class BinanceDataClient:
    """Enhanced Binance client for backtesting with multiple pairs"""
    
    def __init__(self):
        self.base_url = "https://api.binance.com"
        self.session = None
    
    async def init_session(self):
        """Initialize aiohttp session"""
        if self.session is None:
            connector = aiohttp.TCPConnector(limit=100, limit_per_host=30)
            timeout = aiohttp.ClientTimeout(total=30)
            self.session = aiohttp.ClientSession(connector=connector, timeout=timeout)
    
    async def close_session(self):
        """Close aiohttp session"""
        if self.session:
            await self.session.close()
            self.session = None
    
    async def get_exchange_info(self):
        """Get exchange information and active symbols"""
        await self.init_session()
        
        url = f"{self.base_url}/api/v3/exchangeInfo"
        try:
            async with self.session.get(url) as response:
                data = await response.json()
                
                symbols = [
                    item['symbol'] for item in data['symbols']
                    if item['symbol'].endswith('USDT') and item['status'] == 'TRADING'
                ]
                
                logger.info(f"Found {len(symbols)} active USDT trading pairs")
                return symbols
                
        except Exception as e:
            logger.error(f"Error fetching exchange info: {e}")
            return []
    
    async def get_top_volume_symbols(self, limit=20):
        """Get top volume USDT pairs"""
        await self.init_session()
        
        url = f"{self.base_url}/api/v3/ticker/24hr"
        try:
            async with self.session.get(url) as response:
                data = await response.json()
                
                # Filter USDT pairs with good volume
                usdt_pairs = [
                    {
                        'symbol': item['symbol'],
                        'volume': float(item['quoteVolume']),
                        'count': int(item['count'])
                    }
                    for item in data 
                    if (item['symbol'].endswith('USDT') and 
                        item['status'] == 'TRADING' and
                        float(item['quoteVolume']) > 5000000 and  # Min $5M volume
                        int(item['count']) > 10000)  # Min trade count
                ]
                
                # Sort by volume
                usdt_pairs.sort(key=lambda x: x['volume'], reverse=True)
                
                symbols = [pair['symbol'] for pair in usdt_pairs[:limit]]
                volumes = [pair['volume'] for pair in usdt_pairs[:limit]]
                
                logger.info(f"Selected {len(symbols)} top volume pairs")
                logger.info(f"Volume range: ${volumes[-1]:,.0f} - ${volumes[0]:,.0f}")
                
                return symbols
                
        except Exception as e:
            logger.error(f"Error fetching volume data: {e}")
            return SELECTED_SYMBOLS[:limit]  # Fallback
    
    async def fetch_klines(self, symbol, interval=TIMEFRAME, limit=BARS_TO_FETCH):
        """Fetch historical kline data for a symbol"""
        await self.init_session()
        
        url = f"{self.base_url}/api/v3/klines"
        params = {
            'symbol': symbol,
            'interval': interval,
            'limit': limit
        }
        
        try:
            async with self.session.get(url, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    
                    if isinstance(data, list) and len(data) > 0:
                        # Convert to DataFrame
                        columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                                  'close_time', 'quote_asset_volume', 'number_of_trades', 
                                  'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
                        
                        df = pd.DataFrame(data, columns=columns)
                        
                        # Convert numeric columns
                        for col in ['open', 'high', 'low', 'close', 'volume']:
                            df[col] = pd.to_numeric(df[col], errors='coerce')
                        
                        # Convert timestamp
                        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                        df.set_index('timestamp', inplace=True)
                        
                        # Keep only OHLCV
                        df = df[['open', 'high', 'low', 'close', 'volume']].copy()
                        df = df.sort_index()
                        
                        # Basic data validation
                        if len(df) < 100:  # Need minimum data
                            logger.warning(f"Insufficient data for {symbol}: {len(df)} bars")
                            return None
                        
                        # Check for invalid prices
                        if df['close'].isna().any() or (df['close'] <= 0).any():
                            logger.warning(f"Invalid price data for {symbol}")
                            return None
                        
                        logger.info(f"✓ {symbol}: {len(df)} bars ({df.index[0].date()} to {df.index[-1].date()})")
                        return df
                        
                else:
                    logger.error(f"HTTP {response.status} for {symbol}")
                    return None
                    
        except asyncio.TimeoutError:
            logger.error(f"Timeout fetching {symbol}")
            return None
        except Exception as e:
            logger.error(f"Error fetching {symbol}: {e}")
            return None

# ═══════════════════════════════════════════════════════════════════════════════
# VS Wakeup Strategy Implementation
# ═══════════════════════════════════════════════════════════════════════════════

def detect_vs_wakeup_signal(df, check_bar=-1):
    """
    VS Wakeup strategy using actual project implementations
    
    Args:
        df: OHLCV DataFrame
        check_bar: Bar index to check (-1 for last bar)
    
    Returns:
        tuple: (detected, result_dict)
    """
    if len(df) < 50:  # Need sufficient data
        return False, {}
    
    try:
        # Check consolidation (ongoing pattern, no breakout)
        cons_detected, cons_result = detect_consolidation(df, check_bar=check_bar)
        
        if not cons_detected:
            return False, {'reason': 'no_consolidation'}
        
        # Ensure it's not a breakout
        if cons_result.get('breakout', False):
            return False, {'reason': 'consolidation_breakout'}
        
        # Check confluence (wakeup signal within consolidation)
        conf_detected, conf_result = detect_confluence(df, check_bar=check_bar, only_wakeup=True)
        
        if not conf_detected:
            return False, {'reason': 'no_confluence'}
        
        # Combine results for VS Wakeup signal
        idx = check_bar if check_bar >= 0 else len(df) + check_bar
        
        result = {
            'timestamp': df.index[idx],
            'signal_type': 'vs_wakeup',
            'symbol': None,  # Will be set by caller
            'close_price': df.iloc[idx]['close'],
            'volume_usd': df.iloc[idx]['volume'] * df.iloc[idx]['close'],
            'direction': conf_result.get('direction', 'Up'),
            
            # Consolidation info
            'box_age': cons_result.get('box_age', 0),
            'consolidation_height_pct': cons_result.get('height_pct', 0),
            
            # Confluence info  
            'volume_ratio': conf_result.get('volume_ratio', 1.0),
            'momentum_score': conf_result.get('momentum_score', 0),
            'close_off_low': conf_result.get('close_off_low', 50.0),
            'extreme_volume': conf_result.get('extreme_volume', False),
            'extreme_spread': conf_result.get('extreme_spread', False),
            
            # Technical details
            'bar_index': idx,
            'current_bar': (idx == len(df) - 1)
        }
        
        return True, result
        
    except Exception as e:
        logger.error(f"Error in VS Wakeup detection: {e}")
        return False, {'reason': f'error: {str(e)}'}

# ═══════════════════════════════════════════════════════════════════════════════
# Performance Analysis Functions
# ═══════════════════════════════════════════════════════════════════════════════

def find_pivot_high(df, start_idx, lookback=LOOKBACK_BARS, min_bars_ahead=10):
    """
    Find the highest pivot high after a signal with lookback confirmation
    
    Args:
        df: Price DataFrame
        start_idx: Index where signal occurred  
        lookback: Bars to look back for pivot confirmation
        min_bars_ahead: Minimum bars to look ahead
    
    Returns:
        dict: Pivot high analysis results
    """
    if start_idx >= len(df) - min_bars_ahead:
        return {
            'pivot_high_price': np.nan,
            'pivot_high_index': np.nan,
            'pivot_high_date': None,
            'bars_to_pivot': np.nan,
            'gain_to_pivot_pct': np.nan
        }
    
    signal_price = df.iloc[start_idx]['close']
    search_start = start_idx + 1
    search_end = len(df)
    
    best_pivot_high = signal_price
    best_pivot_idx = start_idx
    
    # Search for pivot highs
    for i in range(search_start + lookback, search_end - lookback):
        current_high = df.iloc[i]['high']
        
        # Check if this is a pivot high (higher than lookback bars on both sides)
        is_pivot = True
        
        # Check left side (lookback bars)
        for j in range(i - lookback, i):
            if df.iloc[j]['high'] >= current_high:
                is_pivot = False
                break
        
        # Check right side (lookback bars)
        if is_pivot:
            for j in range(i + 1, min(i + lookback + 1, search_end)):
                if df.iloc[j]['high'] >= current_high:
                    is_pivot = False
                    break
        
        # Update best pivot if this is higher
        if is_pivot and current_high > best_pivot_high:
            best_pivot_high = current_high
            best_pivot_idx = i
    
    # Calculate metrics
    bars_to_pivot = best_pivot_idx - start_idx
    gain_pct = ((best_pivot_high - signal_price) / signal_price * 100) if signal_price > 0 else 0
    
    return {
        'pivot_high_price': best_pivot_high,
        'pivot_high_index': best_pivot_idx,
        'pivot_high_date': df.index[best_pivot_idx] if best_pivot_idx < len(df) else None,
        'bars_to_pivot': bars_to_pivot,
        'gain_to_pivot_pct': gain_pct
    }

def calculate_max_drawdown(df, start_idx, end_idx):
    """
    Calculate maximum drawdown from signal to pivot high, and highest gain before drawdown
    
    Args:
        df: Price DataFrame
        start_idx: Signal bar index
        end_idx: Target bar index (pivot high)
    
    Returns:
        dict: Enhanced drawdown analysis
    """
    if start_idx >= end_idx or end_idx >= len(df):
        return {
            'max_drawdown_pct': 0.0,
            'max_drawdown_price': df.iloc[start_idx]['close'],
            'max_drawdown_index': start_idx,
            'max_drawdown_date': df.index[start_idx],
            'bars_to_max_dd': 0,
            'highest_gain_before_dd_pct': 0.0,
            'highest_gain_before_dd_price': df.iloc[start_idx]['close'],
            'highest_gain_before_dd_date': df.index[start_idx],
            'stopped_before_target': False
        }
    
    signal_price = df.iloc[start_idx]['close']
    
    max_drawdown_pct = 0.0
    max_drawdown_price = signal_price
    max_drawdown_idx = start_idx
    
    highest_price_before_dd = signal_price
    highest_gain_before_dd_pct = 0.0
    highest_gain_before_dd_idx = start_idx
    
    # Track running high and drawdown
    running_high = signal_price
    
    for i in range(start_idx, end_idx + 1):
        current_high = df.iloc[i]['high']
        current_low = df.iloc[i]['low']
        
        # Update running high
        if current_high > running_high:
            running_high = current_high
        
        # Calculate drawdown from running high (not just signal price)
        drawdown_from_high = ((running_high - current_low) / running_high * 100) if running_high > 0 else 0
        drawdown_from_signal = ((signal_price - current_low) / signal_price * 100) if signal_price > 0 else 0
        
        # Track maximum drawdown from signal price
        if drawdown_from_signal > max_drawdown_pct:
            # Before updating max drawdown, capture the highest gain up to this point
            highest_price_before_dd = running_high
            highest_gain_before_dd_pct = ((running_high - signal_price) / signal_price * 100) if signal_price > 0 else 0
            highest_gain_before_dd_idx = i
            
            # Update max drawdown
            max_drawdown_pct = drawdown_from_signal
            max_drawdown_price = current_low
            max_drawdown_idx = i
    
    # Check if stopped before reaching target (assuming 10% drawdown stops you out)
    stop_loss_threshold = 10.0
    stopped_before_target = max_drawdown_pct >= stop_loss_threshold
    
    return {
        'max_drawdown_pct': max_drawdown_pct,
        'max_drawdown_price': max_drawdown_price,
        'max_drawdown_index': max_drawdown_idx,
        'max_drawdown_date': df.index[max_drawdown_idx],
        'bars_to_max_dd': max_drawdown_idx - start_idx,
        'highest_gain_before_dd_pct': highest_gain_before_dd_pct,
        'highest_gain_before_dd_price': highest_price_before_dd,
        'highest_gain_before_dd_date': df.index[highest_gain_before_dd_idx],
        'stopped_before_target': stopped_before_target,
        'stop_loss_threshold_used': stop_loss_threshold
    }

def analyze_quick_reversal(df, start_idx, bars_ahead=QUICK_REVERSAL_BARS):
    """
    Analyze quick reversal potential in next N bars
    
    Args:
        df: Price DataFrame
        start_idx: Signal bar index
        bars_ahead: Number of bars to analyze
    
    Returns:
        dict: Quick reversal analysis
    """
    if start_idx >= len(df) - bars_ahead:
        return {
            'quick_reversal_max_high': np.nan,
            'quick_reversal_gain_pct': np.nan,
            'quick_reversal_occurred': False
        }
    
    signal_price = df.iloc[start_idx]['close']
    
    max_high = signal_price
    for i in range(start_idx + 1, min(start_idx + bars_ahead + 1, len(df))):
        max_high = max(max_high, df.iloc[i]['high'])
    
    gain_pct = ((max_high - signal_price) / signal_price * 100) if signal_price > 0 else 0
    quick_reversal = gain_pct >= 2.0  # 2% threshold for quick reversal
    
    return {
        'quick_reversal_max_high': max_high,
        'quick_reversal_gain_pct': gain_pct,
        'quick_reversal_occurred': quick_reversal
    }

# ═══════════════════════════════════════════════════════════════════════════════
# Backtesting Engine
# ═══════════════════════════════════════════════════════════════════════════════

async def backtest_symbol(client, symbol):
    """
    Run backtest on a single symbol
    
    Args:
        client: BinanceDataClient instance
        symbol: Trading pair symbol
    
    Returns:
        list: List of signal results
    """
    logger.info(f"Backtesting {symbol}...")
    
    # Fetch data
    df = await client.fetch_klines(symbol, TIMEFRAME, BARS_TO_FETCH)
    if df is None or len(df) < 100:
        logger.warning(f"Skipping {symbol} - insufficient data")
        return []
    
    signals = []
    
    # Scan for VS Wakeup signals (leave last 20 bars for forward analysis)
    scan_end = len(df) - 20
    
    for i in range(50, scan_end):  # Start after 50 bars for indicator warmup
        detected, result = detect_vs_wakeup_signal(df, check_bar=i)
        
        if detected:
            result['symbol'] = symbol
            
            # Performance analysis
            pivot_analysis = find_pivot_high(df, i, LOOKBACK_BARS)
            drawdown_analysis = calculate_max_drawdown(
                df, i, pivot_analysis['pivot_high_index']
            )
            quick_reversal = analyze_quick_reversal(df, i, QUICK_REVERSAL_BARS)
            
            # Combine all analysis
            signal_result = {
                **result,
                **pivot_analysis,
                **drawdown_analysis,
                **quick_reversal
            }
            
            signals.append(signal_result)
            
            logger.info(f"  Signal {len(signals)}: {result['timestamp'].date()} "
                       f"Gain: {pivot_analysis['gain_to_pivot_pct']:.2f}% "
                       f"DD: -{drawdown_analysis['max_drawdown_pct']:.2f}%")
    
    logger.info(f"✓ {symbol}: {len(signals)} signals found")
    return signals

async def run_backtest():
    """
    Main backtest runner for multiple symbols
    """
    print("\nInitializing backtest...")
    
    client = BinanceDataClient()
    await client.init_session()
    
    try:
        # Select symbols based on mode
        if SYMBOL_MODE == 'top_volume':
            symbols = await client.get_top_volume_symbols(20)
        elif SYMBOL_MODE == 'selected':
            symbols = SELECTED_SYMBOLS
        elif SYMBOL_MODE == 'all_major':
            symbols = MAJOR_SYMBOLS
        else:
            symbols = SELECTED_SYMBOLS
        
        print(f"\nSelected {len(symbols)} symbols for backtesting:")
        print(f"{', '.join(symbols)}")
        print(f"Estimated time: ~{len(symbols) * 2} seconds\n")
        
        # Run backtests
        all_signals = []
        
        for i, symbol in enumerate(symbols, 1):
            print(f"[{i}/{len(symbols)}] Processing {symbol}...")
            
            try:
                signals = await backtest_symbol(client, symbol)
                all_signals.extend(signals)
                
                # Brief pause to respect API limits
                await asyncio.sleep(0.1)
                
            except Exception as e:
                logger.error(f"Error processing {symbol}: {e}")
                continue
        
        return all_signals
        
    finally:
        await client.close_session()

# ═══════════════════════════════════════════════════════════════════════════════
# Results Analysis and Export
# ═══════════════════════════════════════════════════════════════════════════════

def analyze_results(signals):
    """
    Analyze backtest results and generate summary statistics
    
    Args:
        signals: List of signal dictionaries
    
    Returns:
        dict: Analysis summary
    """
    if not signals:
        return {
            'total_signals': 0,
            'summary': 'No signals found'
        }
    
    df_signals = pd.DataFrame(signals)
    
    # Basic statistics
    total_signals = len(df_signals)
    symbols_with_signals = df_signals['symbol'].nunique()
    
    # Performance metrics
    avg_gain = df_signals['gain_to_pivot_pct'].mean()
    median_gain = df_signals['gain_to_pivot_pct'].median()
    max_gain = df_signals['gain_to_pivot_pct'].max()
    min_gain = df_signals['gain_to_pivot_pct'].min()
    
    # Drawdown metrics
    avg_drawdown = df_signals['max_drawdown_pct'].mean()
    max_drawdown = df_signals['max_drawdown_pct'].max()
    
    # Time metrics
    avg_bars_to_pivot = df_signals['bars_to_pivot'].mean()
    
    # Success metrics
    profitable_signals = (df_signals['gain_to_pivot_pct'] > 0).sum()
    success_rate = profitable_signals / total_signals * 100
    
    # Quick reversal analysis
    quick_reversals = df_signals['quick_reversal_occurred'].sum()
    quick_reversal_rate = quick_reversals / total_signals * 100
    
    return {
        'total_signals': total_signals,
        'symbols_with_signals': symbols_with_signals,
        'success_rate': success_rate,
        'avg_gain_pct': avg_gain,
        'median_gain_pct': median_gain,
        'max_gain_pct': max_gain,
        'min_gain_pct': min_gain,
        'avg_drawdown_pct': avg_drawdown,
        'max_drawdown_pct': max_drawdown,
        'avg_bars_to_pivot': avg_bars_to_pivot,
        'quick_reversal_count': quick_reversals,
        'quick_reversal_rate': quick_reversal_rate,
        'df_signals': df_signals
    }

def save_results_to_csv(signals, filename=None):
    """
    Save backtest results to CSV file
    
    Args:
        signals: List of signal dictionaries
        filename: CSV filename (auto-generated if None)
    
    Returns:
        str: Filename of saved file
    """
    if not signals:
        print("No signals to save")
        return None
    
    df = pd.DataFrame(signals)
    
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"vs_wakeup_backtest_{TIMEFRAME}_{timestamp}.csv"
    
    # Select key columns for CSV export
    export_columns = [
        'symbol', 'timestamp', 'signal_type', 'direction',
        'close_price', 'volume_usd', 'box_age',
        'pivot_high_price', 'pivot_high_date', 'bars_to_pivot', 'gain_to_pivot_pct',
        'max_drawdown_pct', 'max_drawdown_date', 'bars_to_max_dd',
        'highest_gain_before_dd_pct', 'highest_gain_before_dd_price', 'highest_gain_before_dd_date',
        'stopped_before_target', 'stop_loss_threshold_used',
        'quick_reversal_max_high', 'quick_reversal_gain_pct', 'quick_reversal_occurred',
        'volume_ratio', 'momentum_score', 'close_off_low', 'extreme_volume', 'extreme_spread'
    ]
    
    # Filter columns that exist
    available_columns = [col for col in export_columns if col in df.columns]
    df_export = df[available_columns].copy()
    
    # Format timestamp columns
    for col in ['timestamp', 'pivot_high_date', 'max_drawdown_date']:
        if col in df_export.columns:
            df_export[col] = pd.to_datetime(df_export[col]).dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Round numeric columns
    numeric_columns = df_export.select_dtypes(include=[np.number]).columns
    df_export[numeric_columns] = df_export[numeric_columns].round(4)
    
    # Save to CSV
    df_export.to_csv(filename, index=False)
    
    print(f"✓ Results saved to: {filename}")
    print(f"  Total signals: {len(df_export)}")
    print(f"  Columns: {len(df_export.columns)}")
    
    return filename

# ═══════════════════════════════════════════════════════════════════════════════
# Main Execution
# ═══════════════════════════════════════════════════════════════════════════════

async def main():
    """Main execution function"""
    
    print(f"\nStarting VS Wakeup backtest...")
    print(f"Timeframe: {TIMEFRAME} | Bars: {BARS_TO_FETCH} | Mode: {SYMBOL_MODE}")
    
    start_time = datetime.now()
    
    try:
        # Run backtest
        signals = await run_backtest()
        
        # Analyze results
        analysis = analyze_results(signals)
        
        # Print summary
        print("\n" + "=" * 60)
        print("BACKTEST RESULTS SUMMARY")
        print("=" * 60)
        
        if analysis['total_signals'] > 0:
            print(f"Total signals found: {analysis['total_signals']}")
            print(f"Symbols with signals: {analysis['symbols_with_signals']}")
            print(f"Success rate: {analysis['success_rate']:.1f}%")
            print(f"Average gain: {analysis['avg_gain_pct']:.2f}%")
            print(f"Median gain: {analysis['median_gain_pct']:.2f}%")
            print(f"Best gain: {analysis['max_gain_pct']:.2f}%")
            print(f"Worst result: {analysis['min_gain_pct']:.2f}%")
            print(f"Average drawdown: {analysis['avg_drawdown_pct']:.2f}%")
            print(f"Max drawdown: {analysis['max_drawdown_pct']:.2f}%")
            print(f"Avg bars to pivot: {analysis['avg_bars_to_pivot']:.1f}")
            print(f"Quick reversals: {analysis['quick_reversal_count']} ({analysis['quick_reversal_rate']:.1f}%)")
            
            # Save to CSV
            csv_file = save_results_to_csv(signals)
            
            # Top performers - show more comprehensive metrics
            df = analysis['df_signals']
            top_signals = df.nlargest(5, 'gain_to_pivot_pct')[
                ['symbol', 'timestamp', 'gain_to_pivot_pct', 'max_drawdown_pct', 
                 'highest_gain_before_dd_pct', 'stopped_before_target', 'bars_to_pivot']
            ]
            print(f"\nTop 5 performing signals:")
            print("Symbol    Date        Total_Gain%  Max_DD%  Gain_Before_DD%  Stopped  Days_to_Peak")
            print("-" * 80)
            for _, row in top_signals.iterrows():
                stopped_status = "YES" if row.get('stopped_before_target', False) else "NO"
                print(f"{row['symbol']:<8} {row['timestamp'].date()!s:<11} "
                      f"{row['gain_to_pivot_pct']:>8.1f}%  {row['max_drawdown_pct']:>6.1f}%  "
                      f"{row.get('highest_gain_before_dd_pct', 0):>12.1f}%   {stopped_status:<7} "
                      f"{row.get('bars_to_pivot', 0):>6.0f}")
            
            # Risk analysis
            stopped_count = df['stopped_before_target'].sum() if 'stopped_before_target' in df.columns else 0
            stopped_rate = (stopped_count / len(df) * 100) if len(df) > 0 else 0
            avg_gain_before_dd = df['highest_gain_before_dd_pct'].mean() if 'highest_gain_before_dd_pct' in df.columns else 0
            
            print(f"\nRisk Analysis:")
            print(f"Signals stopped out (10% DD): {stopped_count} ({stopped_rate:.1f}%)")
            print(f"Average gain before max DD: {avg_gain_before_dd:.2f}%")
            
            # Symbol performance summary
            symbol_summary = df.groupby('symbol').agg({
                'gain_to_pivot_pct': ['count', 'mean', 'max'],
                'max_drawdown_pct': 'mean',
                'highest_gain_before_dd_pct': 'mean',
                'stopped_before_target': 'sum'
            }).round(2)
            symbol_summary.columns = ['Signal_Count', 'Avg_Total_Gain_%', 'Best_Total_Gain_%', 
                                    'Avg_DD_%', 'Avg_Gain_Before_DD_%', 'Stopped_Count']
            
            print(f"\nSymbol Performance Summary:")
            print(symbol_summary.sort_values('Avg_Gain_%', ascending=False).head(10).to_string())
            
        else:
            print("No VS Wakeup signals found in the backtested data")
            print("This could indicate:")
            print("- Strategy parameters may be too strict")
            print("- Selected timeframe may not be optimal for this strategy")  
            print("- Market conditions in the backtested period may not favor this pattern")
        
        end_time = datetime.now()
        duration = end_time - start_time
        print(f"\nBacktest completed in {duration.total_seconds():.1f} seconds")
        
        return signals
        
    except Exception as e:
        logger.error(f"Backtest failed: {e}")
        print(f"Error: {e}")
        return []

# ═══════════════════════════════════════════════════════════════════════════════
# Execution Cell
# ═══════════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    # Run the backtest
    signals = await main()
    
    print(f"\nBacktest complete! Found {len(signals)} signals.")
    
    if signals:
        print(f"\nTo analyze further:")
        print(f"- CSV file contains all signal details")
        print(f"- Check 'gain_to_pivot_pct' for profitability")
        print(f"- Review 'max_drawdown_pct' for risk assessment")
        print(f"- Examine 'quick_reversal_occurred' for short-term opportunities")
        
        # Quick data access
        df_results = pd.DataFrame(signals)
        
        print(f"\nResults DataFrame available as 'df_results'")
        print(f"Shape: {df_results.shape}")
        print(f"Columns: {list(df_results.columns)}")
        
        # Show sample data
        print(f"\nSample results:")
        sample_cols = ['symbol', 'timestamp', 'direction', 'gain_to_pivot_pct', 'max_drawdown_pct', 'quick_reversal_occurred']
        available_cols = [col for col in sample_cols if col in df_results.columns]
        print(df_results[available_cols].head().to_string(index=False))
    
    else:
        print(f"\nNo signals found. Consider adjusting:")
        print(f"- TIMEFRAME (try '4h' or '1h' for more frequent signals)")
        print(f"- SYMBOL_MODE (try 'all_major' for broader coverage)")  
        print(f"- Strategy parameters in detect_consolidation/detect_confluence")

# ═══════════════════════════════════════════════════════════════════════════════
# Additional Analysis Functions (Optional Usage)
# ═══════════════════════════════════════════════════════════════════════════════

def plot_signal_distribution(df_signals):
    """
    Plot distribution of signal performance (requires matplotlib)
    
    Usage: plot_signal_distribution(df_results)
    """
    try:
        import matplotlib.pyplot as plt
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Gain distribution
        axes[0,0].hist(df_signals['gain_to_pivot_pct'], bins=20, alpha=0.7, color='green')
        axes[0,0].axvline(df_signals['gain_to_pivot_pct'].mean(), color='red', linestyle='--', label='Mean')
        axes[0,0].set_title('Gain to Pivot Distribution')
        axes[0,0].set_xlabel('Gain %')
        axes[0,0].legend()
        
        # Drawdown distribution
        axes[0,1].hist(df_signals['max_drawdown_pct'], bins=20, alpha=0.7, color='red')
        axes[0,1].axvline(df_signals['max_drawdown_pct'].mean(), color='blue', linestyle='--', label='Mean')
        axes[0,1].set_title('Max Drawdown Distribution')
        axes[0,1].set_xlabel('Drawdown %')
        axes[0,1].legend()
        
        # Bars to pivot
        axes[1,0].hist(df_signals['bars_to_pivot'], bins=20, alpha=0.7, color='blue')
        axes[1,0].set_title('Bars to Pivot High Distribution')
        axes[1,0].set_xlabel('Bars')
        
        # Gain vs Drawdown scatter
        axes[1,1].scatter(df_signals['max_drawdown_pct'], df_signals['gain_to_pivot_pct'], alpha=0.6)
        axes[1,1].set_xlabel('Max Drawdown %')
        axes[1,1].set_ylabel('Gain to Pivot %')
        axes[1,1].set_title('Risk vs Reward')
        
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("matplotlib not available. Install with: pip install matplotlib")

def analyze_by_symbol(df_signals):
    """
    Detailed analysis by symbol
    
    Usage: analyze_by_symbol(df_results)
    """
    symbol_analysis = df_signals.groupby('symbol').agg({
        'gain_to_pivot_pct': ['count', 'mean', 'median', 'std', 'min', 'max'],
        'max_drawdown_pct': ['mean', 'max'],
        'bars_to_pivot': 'mean',
        'quick_reversal_occurred': ['sum', 'mean']
    }).round(3)
    
    # Flatten column names
    symbol_analysis.columns = ['_'.join(col).strip() for col in symbol_analysis.columns]
    
    # Add success rate
    symbol_analysis['success_rate'] = (
        df_signals.groupby('symbol')['gain_to_pivot_pct'].apply(lambda x: (x > 0).mean() * 100)
    ).round(1)
    
    # Sort by average gain
    symbol_analysis = symbol_analysis.sort_values('gain_to_pivot_pct_mean', ascending=False)
    
    print("Detailed Symbol Analysis:")
    print(symbol_analysis.to_string())
    
    return symbol_analysis

def export_for_tradingview(df_signals, symbol='BTCUSDT'):
    """
    Export signals for a specific symbol in TradingView format
    
    Usage: export_for_tradingview(df_results, 'BTCUSDT')
    """
    symbol_signals = df_signals[df_signals['symbol'] == symbol].copy()
    
    if len(symbol_signals) == 0:
        print(f"No signals found for {symbol}")
        return
    
    # Format for TradingView
    tv_format = symbol_signals[['timestamp', 'close_price', 'direction', 'gain_to_pivot_pct']].copy()
    tv_format['timestamp'] = tv_format['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    filename = f"vs_wakeup_{symbol}_tradingview.csv"
    tv_format.to_csv(filename, index=False)
    
    print(f"TradingView format exported to: {filename}")
    print(f"Signals for {symbol}: {len(tv_format)}")
    
    return tv_format

# ═══════════════════════════════════════════════════════════════════════════════
# Usage Instructions
# ═══════════════════════════════════════════════════════════════════════════════

print(f"""
Usage Instructions:
==================

1. Run the main backtest:
   signals = await main()

2. Analyze results:
   df_results = pd.DataFrame(signals)
   
3. Optional analysis functions:
   - plot_signal_distribution(df_results)  # Requires matplotlib
   - analyze_by_symbol(df_results)         # Detailed symbol breakdown
   - export_for_tradingview(df_results, 'BTCUSDT')  # Export for TV

4. Configuration options (modify at top):
   - TIMEFRAME: '1d', '4h', '1h' (Binance supported intervals)
   - SYMBOL_MODE: 'top_volume', 'selected', 'all_major'  
   - BARS_TO_FETCH: Number of historical bars (max 1000)

5. CSV output contains:
   - Signal details (timestamp, symbol, direction)
   - Performance metrics (gain to pivot, max drawdown)
   - Technical indicators (volume ratio, momentum score)
   - Quick reversal analysis (3-bar forward look)

Note: Ensure you're in the project directory with custom_strategies available.
""")

In [ ]:
#TEST ANY PAIR ON ANY EXCHANGE ON ANY TIMEFRAME

#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Real-Time Strategy Checker - All Strategies Scanner
===================================================

Interactive notebook to check all native and composed strategies against any trading pair
from your project workspace. Tests both current and last closed bars.

Strategies Tested:
- Native: confluence, consolidation_breakout, channel_breakout, loaded_bar, 
         trend_breakout, pin_up, sma50_breakout
- Composed: hbs_breakout, vs_wakeup
- Futures-only: reversal_bar, pin_down

Usage:
1. Set SYMBOL, EXCHANGE, TIMEFRAME variables
2. Run the scan
3. Get ✅❌ results for all strategies

Author: Market Scanner v2.10
Date: 2024
"""

import pandas as pd
import numpy as np
import asyncio
import logging
import sys
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add project path
project_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_dir)

print("Real-Time Strategy Checker - All Strategies Scanner")
print("=" * 60)

# ═══════════════════════════════════════════════════════════════════════════════
# Configuration - MODIFY THESE VALUES
# ═══════════════════════════════════════════════════════════════════════════════

# Trading pair to analyze
SYMBOL = "XAIUSDT"

# Exchange to use
EXCHANGE = "binance_spot"  # Options: binance_spot, binance_futures, bybit_spot, etc.

# Timeframe to analyze  
TIMEFRAME = "1w"  # Options: 1h, 4h, 1d, 2d, 3d, 4d, 1w

# ═══════════════════════════════════════════════════════════════════════════════
# Import Project Components
# ═══════════════════════════════════════════════════════════════════════════════

try:
    # Import exchange clients
    from exchanges import (
        BinanceSpotClient, BinanceFuturesClient, BybitSpotClient, BybitFuturesClient,
        GateioSpotClient, GateioFuturesClient, KucoinSpotClient, MexcSpotClient, MexcFuturesClient
    )
    
    # Import all available custom strategies from your project
    from custom_strategies import (
        detect_volume_surge, detect_weak_uptrend, detect_pin_down, detect_confluence,
        detect_consolidation, detect_channel, detect_consolidation_breakout,
        detect_channel_breakout, detect_wedge_breakout, detect_sma50_breakout,
        detect_trend_breakout, detect_pin_up
    )
    
    # Import VSA strategies
    vsa_available = []
    try:
        from breakout_vsa.strategies.loaded_bar import get_params as get_loaded_bar_params
        from breakout_vsa.strategies.reversal_bar import get_params as get_reversal_bar_params
        from breakout_vsa.core import vsa_detector
        vsa_available.extend(['loaded_bar', 'reversal_bar'])
    except ImportError:
        print("Note: VSA strategies not available")
    
    print(f"✓ Successfully imported all project components")
    
except ImportError as e:
    print(f"Import error: {e}")
    sys.exit(1)

# Strategy Definitions based on your actual project structure
NATIVE_STRATEGIES = [
    'confluence',
    'consolidation_breakout', 
    'channel_breakout',
    'loaded_bar',
    'trend_breakout',
    'pin_up',
    'sma50_breakout'
]

COMPOSED_STRATEGIES = [
    'hbs_breakout',
    'vs_wakeup'
]

FUTURES_ONLY_STRATEGIES = [
    'reversal_bar',
    'pin_down'
]

ALL_STRATEGIES = NATIVE_STRATEGIES + COMPOSED_STRATEGIES + FUTURES_ONLY_STRATEGIES

# ═══════════════════════════════════════════════════════════════════════════════
# Data Fetching
# ═══════════════════════════════════════════════════════════════════════════════

async def fetch_market_data(symbol, exchange, timeframe):
    """
    Fetch market data using project exchange clients
    
    Args:
        symbol: Trading pair symbol
        exchange: Exchange name
        timeframe: Timeframe string
    
    Returns:
        DataFrame: OHLCV data or None if error
    """
    if exchange not in EXCHANGE_CLIENTS:
        return None
    
    try:
        # Initialize exchange client
        ClientClass = EXCHANGE_CLIENTS[exchange]
        client = ClientClass(timeframe=timeframe)
        
        await client.init_session()
        df = await client.fetch_klines(symbol)
        await client.close_session()
        
        if df is not None and len(df) > 50:
            return df
        else:
            return None
            
    except Exception as e:
        return None

# ═══════════════════════════════════════════════════════════════════════════════
# Strategy Testing Functions
# ═══════════════════════════════════════════════════════════════════════════════

def test_native_strategy(strategy_name, df, check_bar):
    """Test a native strategy"""
    try:
        if strategy_name == 'confluence':
            detected, result = detect_confluence(df, check_bar=check_bar)
        elif strategy_name == 'consolidation_breakout':
            detected, result = detect_consolidation_breakout(df, check_bar=check_bar)
        elif strategy_name == 'channel_breakout':
            detected, result = detect_channel_breakout(df, check_bar=check_bar)
        elif strategy_name == 'trend_breakout':
            detected, result = detect_trend_breakout(df, check_bar=check_bar)
        elif strategy_name == 'pin_up':
            detected, result = detect_pin_up(df, check_bar=check_bar)
        elif strategy_name == 'sma50_breakout':
            detected, result = detect_sma50_breakout(df, check_bar=check_bar)
        elif strategy_name == 'loaded_bar':
            # VSA loaded bar
            params = get_loaded_bar_params()
            condition, vsa_result = vsa_detector(df, params)
            detected = condition.iloc[check_bar] if len(condition) > abs(check_bar) else False
            result = {'vsa_type': 'loaded_bar'} if detected else {}
        else:
            return False, {'error': f'Unknown native strategy: {strategy_name}'}
        
        return detected, result
        
    except Exception as e:
        return False, {'error': str(e)}

def test_composed_strategy(strategy_name, df, check_bar):
    """Test a composed strategy"""
    try:
        if strategy_name == 'hbs_breakout':
            # HBS = (Consolidation Breakout OR Channel Breakout) + Confluence
            cb_detected, cb_result = detect_consolidation_breakout(df, check_bar=check_bar)
            chb_detected, chb_result = detect_channel_breakout(df, check_bar=check_bar)
            conf_detected, conf_result = detect_confluence(df, check_bar=check_bar)
            
            breakout_detected = cb_detected or chb_detected
            hbs_detected = breakout_detected and conf_detected
            
            if hbs_detected:
                result = {
                    'hbs_type': 'consolidation_breakout' if cb_detected else 'channel_breakout',
                    'has_confluence': True,
                    'consolidation_detected': cb_detected,
                    'channel_breakout_detected': chb_detected
                }
            else:
                result = {
                    'consolidation_detected': cb_detected,
                    'channel_breakout_detected': chb_detected,
                    'confluence_detected': conf_detected,
                    'reason': 'missing_component'
                }
            
            return hbs_detected, result
            
        elif strategy_name == 'vs_wakeup':
            
            # VS Wakeup = Consolidation (ongoing) + Confluence
            cons_detected, cons_result = detect_consolidation(df, check_bar=check_bar)
            conf_detected, conf_result = detect_confluence(df, check_bar=check_bar, only_wakeup=True)
            
            # Must be consolidation without breakout
            is_consolidation = cons_detected and not cons_result.get('breakout', False)
            vs_detected = is_consolidation and conf_detected
            
            result = {
                'consolidation_detected': cons_detected,
                'consolidation_breakout': cons_result.get('breakout', False),
                'confluence_detected': conf_detected,
                'vs_wakeup_valid': vs_detected
            }
            
            return vs_detected, result
            
        else:
            return False, {'error': f'Unknown composed strategy: {strategy_name}'}
            
    except Exception as e:
        return False, {'error': str(e)}

def test_futures_strategy(strategy_name, df, check_bar):
    """Test a futures-only strategy"""
    try:
        if strategy_name == 'reversal_bar':
            # VSA reversal bar
            params = get_reversal_bar_params()
            condition, vsa_result = vsa_detector(df, params)
            detected = condition.iloc[check_bar] if len(condition) > abs(check_bar) else False
            result = {'vsa_type': 'reversal_bar'} if detected else {}
            
        elif strategy_name == 'pin_down':
            # Pin down pattern
            from custom_strategies import detect_pin_down
            detected, result = detect_pin_down(df, check_bar=check_bar)
            
        else:
            return False, {'error': f'Unknown futures strategy: {strategy_name}'}
        
        return detected, result
        
    except Exception as e:
        return False, {'error': str(e)}

def test_single_strategy(strategy_name, df, check_bar):
    """Test any strategy by name"""
    if strategy_name in NATIVE_STRATEGIES:
        return test_native_strategy(strategy_name, df, check_bar)
    elif strategy_name in COMPOSED_STRATEGIES:
        return test_composed_strategy(strategy_name, df, check_bar)
    elif strategy_name in FUTURES_ONLY_STRATEGIES:
        return test_futures_strategy(strategy_name, df, check_bar)
    else:
        return False, {'error': f'Unknown strategy: {strategy_name}'}

# ═══════════════════════════════════════════════════════════════════════════════
# Main Scanner Function
# ═══════════════════════════════════════════════════════════════════════════════

async def scan_all_strategies(symbol, exchange, timeframe):
    """
    Scan all strategies for a given symbol/exchange/timeframe
    
    Returns:
        dict: Results for current and last closed bars
    """
    # Fetch data
    df = await fetch_market_data(symbol, exchange, timeframe)
    if df is None:
        return None
    
    # Check if futures strategies should be included
    is_futures = 'futures' in exchange.lower()
    strategies_to_test = ALL_STRATEGIES if is_futures else NATIVE_STRATEGIES + COMPOSED_STRATEGIES
    
    results = {
        'symbol': symbol,
        'exchange': exchange,
        'timeframe': timeframe,
        'scan_time': datetime.now(),
        'latest_price': df['close'].iloc[-1],
        'current_bar': {},
        'last_closed_bar': {},
        'summary': {}
    }
    
    # Test current bar (-1)
    current_signals = 0
    for strategy in strategies_to_test:
        detected, result = test_single_strategy(strategy, df, check_bar=-1)
        results['current_bar'][strategy] = {
            'detected': detected,
            'result': result
        }
        if detected:
            current_signals += 1
    
    # Test last closed bar (-2)
    closed_signals = 0
    for strategy in strategies_to_test:
        detected, result = test_single_strategy(strategy, df, check_bar=-2)
        results['last_closed_bar'][strategy] = {
            'detected': detected,
            'result': result
        }
        if detected:
            closed_signals += 1
    
    # Summary
    results['summary'] = {
        'total_strategies_tested': len(strategies_to_test),
        'current_bar_signals': current_signals,
        'last_closed_bar_signals': closed_signals,
        'any_signals': current_signals > 0 or closed_signals > 0
    }
    
    return results

# ═══════════════════════════════════════════════════════════════════════════════
# Results Display Functions
# ═══════════════════════════════════════════════════════════════════════════════

def display_results_summary(results):
    """Display formatted results summary"""
    if results is None:
        print("No results to display")
        return
    
    # Calculate separate counts for native and composed strategies
    native_current = sum(1 for s in NATIVE_STRATEGIES if results['current_bar'][s]['detected'])
    native_closed = sum(1 for s in NATIVE_STRATEGIES if results['last_closed_bar'][s]['detected'])
    composed_current = sum(1 for s in COMPOSED_STRATEGIES if results['current_bar'][s]['detected'])
    composed_closed = sum(1 for s in COMPOSED_STRATEGIES if results['last_closed_bar'][s]['detected'])
    
    print(f"{results['symbol']} | {results['timeframe']} | {results['exchange']} | Latest Price: ${results['latest_price']:,.2f}")
    print()
    print(f"{'Strategy':<22} {'Current Bar (' + str(native_current) + ')':<18} {'Last Closed (' + str(native_closed) + ')':<18} {'Category':<12}")
    print("-" * 72)
    
    # Native strategies first
    for strategy in NATIVE_STRATEGIES:
        if strategy in results['current_bar']:
            current = "✅" if results['current_bar'][strategy]['detected'] else "❌"
            closed = "✅" if results['last_closed_bar'][strategy]['detected'] else "❌"
            print(f"{strategy:<22} {current:<18} {closed:<18} {'Native':<12}")
    
    # Add separator line and header for composed strategies
    print("-" * 72)
    print(f"{'Strategy':<22} {'Current Bar (' + str(composed_current) + ')':<18} {'Last Closed (' + str(composed_closed) + ')':<18} {'Category':<12}")
    print("-" * 72)
    
    # Composed strategies
    for strategy in COMPOSED_STRATEGIES:
        if strategy in results['current_bar']:
            current = "✅" if results['current_bar'][strategy]['detected'] else "❌"
            closed = "✅" if results['last_closed_bar'][strategy]['detected'] else "❌"
            print(f"{strategy:<22} {current:<18} {closed:<18} {'Composed':<12}")

def display_detailed_results(results):
    """This function is now merged with display_results_summary"""
    pass

def display_signal_details(results):
    """Display details for detected signals - REMOVED"""
    pass

# Main Execution
# ═══════════════════════════════════════════════════════════════════════════════

async def main():
    """Main execution function"""
    try:
        # Run the scan
        results = await scan_all_strategies(SYMBOL, EXCHANGE, TIMEFRAME)
        
        if results:
            # Display results - single clean table
            display_results_summary(results)
            
            # Save results to variable for further analysis
            global scan_results
            scan_results = results
            
        else:
            print("Scan failed - no results generated")
            
    except Exception as e:
        print(f"Scan error: {e}")
        logging.error(f"Main execution error: {e}")

# Quick Configuration Functions
async def quick_scan(symbol, exchange="binance_spot", timeframe="1d"):
    """
    Quick scan function for easy testing
    
    Usage:
        await quick_scan("ETHUSDT", "binance_spot", "4h")
    """
    global SYMBOL, EXCHANGE, TIMEFRAME
    SYMBOL = symbol
    EXCHANGE = exchange  
    TIMEFRAME = timeframe
    
    print(f"Quick scan: {symbol} on {exchange} ({timeframe})")
    await main()

def list_available_exchanges():
    """List all available exchanges"""
    print("Available exchanges:")
    for exchange in sorted(EXCHANGE_CLIENTS.keys()):
        print(f"  • {exchange}")

def list_all_strategies():
    """List all available strategies by category"""
    print("Available strategies:")
    print(f"\nNative Strategies ({len(NATIVE_STRATEGIES)}):")
    for strategy in NATIVE_STRATEGIES:
        print(f"  • {strategy}")
    
    print(f"\nComposed Strategies ({len(COMPOSED_STRATEGIES)}):")
    for strategy in COMPOSED_STRATEGIES:
        print(f"  • {strategy}")
    
    print(f"\nFutures-Only Strategies ({len(FUTURES_ONLY_STRATEGIES)}):")
    for strategy in FUTURES_ONLY_STRATEGIES:
        print(f"  • {strategy}")

# Run the main scan
if __name__ == "__main__":
    await main()